# Calculations for BBX behavioral data

This code notebook is intended to walkthrough the steps done for the various data calculations for the BBX behavioral dataset.  
Scoring is done based on the survery type, unique calculations can be found in the [data dict](https://docs.google.com/spreadsheets/d/15wvY8vC0pPF71aHHQauibh5d6ZQclbXh/edit#gid=1183265718) google sheet. Heosted here is also variable defnitions and reverse scoring values.  
We translate the *reverse scoring* variables before calculations.


**Input:**  
* FFQ_Food_Database (Chocolate).xlsx
* bbx_msBxScn_datadict_12_30_20.xlsx
  

## TOC:
* [Load Your Data](#dataload)
* [FFQ](#ffq)
* [BIS](#bis)
* [FCI](#fci)
* [DEBQ](#debq)
* [SPSRQ](#spsrq)
* [BIS/BAS](#bisbas)
* [PFS](#pfs)
* [YFAS](#yfas)
* [IPAQ](#ipaq)


In [204]:
import os, glob
import sys
import pandas as pd
import numpy as np
from IPython.display import display

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
pd.set_option("max_rows", None)



In [205]:
# uploading custom package 
sys.path.append('C:\\Users\\19802\\Documents\\nibl\\bbx\\code\\scripts')
import data_dict

## Load Your Data <a class="anchor" id="dataload"></a>


In [206]:
# Set data paths
data_path='C:\\Users\\19802\\Documents\\nibl'#'/Users/nikkibytes/Documents/git_nibl/bbx/data/behavioral/data_dict'

# Grab list of behavioral files found in the data bath (raw and clean data)
s1_behavioral=(os.path.join(data_path, "bbx_inputs", "bbx_w1_msBxScn_01_06_21.csv"))
scoring_file=os.path.join(data_path, 'bbx_inputs', 'bbx_scoring.csv')# 'bbx_msBxScn_datadict_12_30_20.xlsx')
ffq_nutrient_file=os.path.join(data_path, "bbx_inputs", 'nutrient_data_by_ffq_quest.csv')
output_path=os.path.join(data_path, "bbx_outputs")


# Build dataframes from data_dictionary file, 
# scoring and conversion information found in these files
s1_behav=pd.read_csv(s1_behavioral)
s1_behav.set_index("patID", inplace=True)

nutrient_info=pd.read_csv(ffq_nutrient_file)
nutrient_info.columns = nutrient_info.iloc[0]
nutrient_info.set_index('FFQ_quest', inplace=True)
nutrient_info = nutrient_info[1:]

#labels = pd.read_excel(scoring_file,  sheet_name="bbx_codebook")
#labels = labels.loc[:,["LABEL", "NAME", "CODE", "range"]]
scoring = pd.read_csv(scoring_file)#pd.read_excel(scoring_file, sheet_name="scoring")
scoring = scoring.loc[:,["Set", "Output", "Calculation","Description", "Notes"]]
#reverse_scoring = pd.read_excel(scoring_file, sheet_name="reverse_scoring")

**View Dataframes:**

In [207]:
# Load behavioral data
display(s1_behav.head())
#print(s1_behav.columns.values)

,age,gender,w1Bx_ethnicity,w1Bx_race,w1Bx_date,w1Bx_height_1,w1Bx_weight_1,w1Bx_hungry,w1Bx_full,w1Bx_thirsty,w1Bx_nauseous,w1Bx_copleasant,w1Bx_codesirable,w1Bx_cosweet,w1Bx_cointense,w1Bx_cobitter,w1Bx_cofamiliar,w1Bx_ucopleasant,w1Bx_ucodesirable,w1Bx_ucosweet,w1Bx_ucointense,w1Bx_ucobitter,w1Bx_ucofamiliar,w1Bx_slpleasant,w1Bx_sldesirable,w1Bx_slsweet,w1Bx_slintense,w1Bx_slbitter,w1Bx_slfamiliar,w1Bx_uslpleasant,w1Bx_usldesirable,w1Bx_uslsweet,w1Bx_uslintense,w1Bx_uslbitter,w1Bx_uslfamiliar,w1Bx_diwpleasant,w1Bx_diwdesirable,w1Bx_diwsweet,w1Bx_diwintense,w1Bx_diwbitter,w1Bx_diwfamiliar,w1Bx_ffq_1,w1Bx_ffq_2,w1Bx_ffq_3,w1Bx_ffq_4,w1Bx_ffq_5,w1Bx_ffq_6,w1Bx_ffq_7,w1Bx_ffq_8,w1Bx_ffq_9,w1Bx_ffq_10,w1Bx_ffq_11,w1Bx_ffq_12,w1Bx_ffq_13,w1Bx_ffq_14,w1Bx_ffq_15,w1Bx_ffq_16,w1Bx_ffq_17,w1Bx_ffq_18,w1Bx_ffq_19,w1Bx_ffq_20,w1Bx_ffq_21,w1Bx_ffq_22,w1Bx_ffq_23,w1Bx_ffq_24,w1Bx_ffq_25,w1Bx_ffq_26,w1Bx_ffq_27,w1Bx_ffq_28,w1Bx_ffq_29,w1Bx_ffq_30,w1Bx_ffq_31,w1Bx_ffq_32,w1Bx_ffq_33,w1Bx_ffq_34,w1Bx_ffq_35,w1Bx_ffq_36,w1Bx_ffq_37,w1Bx_ffq_38,w1Bx_ffq_39,w1Bx_ffq_40,w1Bx_ffq_41,w1Bx_ffq_42,w1Bx_ffq_43,w1Bx_ffq_44,w1Bx_ffq_45,w1Bx_ffq_46,w1Bx_ffq_47,w1Bx_ffq_48,w1Bx_ffq_49,w1Bx_ffq_50,w1Bx_ffq_51,w1Bx_ffq_52,w1Bx_ffq_53,w1Bx_ffq_54,w1Bx_ffq_55,w1Bx_ffq_56,w1Bx_ffq_57,w1Bx_ffq_58,w1Bx_ffq_59,w1Bx_ffq_60,w1Bx_ffq_61,w1Bx_ffq_62,w1Bx_ffq_63,w1Bx_ffq_64,w1Bx_ffq_65,w1Bx_ffq_66,w1Bx_ffq_67,w1Bx_ffq_68,w1Bx_ffq_69,w1Bx_ffq_70,w1Bx_ffq_71,w1Bx_ffq_72,w1Bx_ffq_73,w1Bx_ffq_74,w1Bx_ffq_75,w1Bx_ffq_76,w1Bx_bis_1,w1Bx_bis_2,w1Bx_bis_3,w1Bx_bis_4,w1Bx_bis_5,w1Bx_bis_6,w1Bx_bis_7,w1Bx_bis_8,w1Bx_bis_9,w1Bx_bis_10,w1Bx_bis_11,w1Bx_bis_12,w1Bx_bis_13,w1Bx_bis_14,w1Bx_bis_15,w1Bx_fciCrv_1,w1Bx_fciCrv_2,w1Bx_fciCrv_3,w1Bx_fciCrv_4,w1Bx_fciCrv_5,w1Bx_fciCrv_6,w1Bx_fciCrv_7,w1Bx_fciCrv_8,w1Bx_fciCrv_9,w1Bx_fciCrv_10,w1Bx_fciCrv_11,w1Bx_fciCrv_12,w1Bx_fciCrv_13,w1Bx_fciCrv_14,w1Bx_fciCrv_15,w1Bx_fciCrv_16,w1Bx_fciCrv_17,w1Bx_fciCrv_18,w1Bx_fciCrv_19,w1Bx_fciCrv_20,w1Bx_fciCrv_21,w1Bx_fciCrv_22,w1Bx_fciCrv_23,w1Bx_fciCrv_24,w1Bx_fciCrv_25,w1Bx_fciCrv_26,w1Bx_fciCrv_27,w1Bx_fciCrv_28,w1Bx_fciLik_1,w1Bx_fciLik_2,w1Bx_fciLik_3,w1Bx_fciLik_4,w1Bx_fciLik_5,w1Bx_fciLik_6,w1Bx_fciLik_7,w1Bx_fciLik_8,w1Bx_fciLik_9,w1Bx_fciLik_10,w1Bx_fciLik_11,w1Bx_fciLik_12,w1Bx_fciLik_13,w1Bx_fciLik_14,w1Bx_fciLik_15,w1Bx_fciLik_16,w1Bx_fciLik_17,w1Bx_fciLik_18,w1Bx_fciLik_19,w1Bx_fciLik_20,w1Bx_fciLik_21,w1Bx_fciLik_22,w1Bx_fciLik_23,w1Bx_fciLik_24,w1Bx_fciLik_25,w1Bx_fciLik_26,w1Bx_fciLik_27,w1Bx_fciLik_28,w1Bx_debq_1,w1Bx_debq_2,w1Bx_debq_3,w1Bx_debq_4,w1Bx_debq_5,w1Bx_debq_6,w1Bx_debq_7,w1Bx_debq_8,w1Bx_debq_9,w1Bx_debq_10,w1Bx_debq_11,w1Bx_debq_12,w1Bx_debq_13,w1Bx_debq_14,w1Bx_debq_15,w1Bx_debq_16,w1Bx_debq_17,w1Bx_debq_18,w1Bx_debq_19,w1Bx_debq_20,w1Bx_debq_21,w1Bx_debq_22,w1Bx_debq_23,w1Bx_debq_24,w1Bx_debq_25,w1Bx_debq_26,w1Bx_debq_27,w1Bx_debq_28,w1Bx_debq_29,w1Bx_debq_30,w1Bx_debq_31,w1Bx_debq_32,w1Bx_debq_33,w1Bx_ipaq_1,w1Bx_ipaq2_5,w1Bx_ipaq2_6,w1Bx_ipaq3_1,w1Bx_ipaq3_2,w1Bx_ipaq4_1,w1Bx_ipaq4_2,w1Bx_ipaq5_1,w1Bx_ipaq5_2,w1Bx_ipaq6_1,w1Bx_ipaq6_2,w1Bx_ipaq7_1,w1Bx_ipaq7_2,w1Bx_ipaq8_1,w1Bx_ipaq8_2,w1Bx_ipaq9_1,w1Bx_ipaq9_2,w1Bx_ipaq10_1,w1Bx_ipaq10_2,w1Bx_ipaq11_1,w1Bx_ipaq11_2,w1Bx_ipaq12_1,w1Bx_ipaq12_2,w1Bx_ipaq13_1,w1Bx_ipaq13_2,w1Bx_ipaq14_1,w1Bx_ipaq14_2,w1Bx_ipaq15_1,w1Bx_ipaq15_2,w1Bx_ipaq16_1,w1Bx_ipaq16_2,w1Bx_ipaq17_1,w1Bx_ipaq17_2,w1Bx_ipaq18_1,w1Bx_ipaq18_2,w1Bx_ipaq19_1,w1Bx_ipaq19_2,w1Bx_ipaq20_1,w1Bx_ipaq20_2,w1Bx_ipaq21_1,w1Bx_ipaq21_2,w1Bx_ipaq22_1,w1Bx_ipaq22_2,w1Bx_ipaq23_1,w1Bx_ipaq23_2,w1Bx_ipaq24_1,w1Bx_ipaq24_2,w1Bx_ipaq25_1,w1Bx_ipaq25_2,w1Bx_hand_1,w1Bx_hand_2,w1Bx_hand_3,w1Bx_hand_4,w1Bx_hand_5,w1Bx_hand_6,w1Bx_hand_7,w1Bx_hand_8,w1Bx_hand_9,w1Bx_hand_10,w1Bx_hand_11,w1Bx_hand_12,w1Bx_spsrq1,w1Bx_spsrq2,w1Bx_spsrq3,w1Bx_spsrq4,w1Bx_spsrq5,w1Bx_spsrq6,w1Bx_spsrq7,w1Bx_spsrq8,w1Bx_spsrq9,w1Bx_spsrq10,w1Bx_spsrq11,w1Bx_spsrq12,w1Bx_spsrq13,w1Bx_spsrq14,w1Bx_spsrq15,w1Bx_spsrq16,w1Bx_spsrq17,w1Bx_spsrq18,w1Bx_spsrq19,w1Bx_spsrq20,w1Bx_yfas_1,w1Bx_yfas

In [208]:
display(nutrient_info.head())
print(nutrient_info.columns.values)

,grams/med portion,kcal/100g,Pro/100g,fat(g)/100g,CHO/100g,sugars/100g,Satfat(g)/g
FFQ_quest,,,,,,,
ffq1,123.3333,66.0,0.5700,0.2133,17.2933,10.7900,0.0540
ffq2,84.7500,47.5,0.7950,0.2000,11.9300,10.0875,0.0513
ffq3,187.5000,88.0,4.9567,0.5133,17.3100,3.3433,0.1100
ffq4,119.0000,91.0,5.4100,1.1700,15.2600,0.4600,0.3910
ffq5,15.0000,23.0,2.8600,0.3900,3.6300,0.4200,0.0600


['grams/med portion ' 'kcal/100g' 'Pro/100g' 'fat(g)/100g' 'CHO/100g'
 'sugars/100g' 'Satfat(g)/g']


In [209]:
#display(labels.head())


In [210]:
display(scoring.head())


,Set,Output,Calculation,Description,Notes
0,FFQ,w1ffq##_kcal,w1Bx_ffq_## * kcal_ffq##,Total kcal consumed for each food variable,kcal_ffq## comes from FFQ excel sheet; * assuming reverse scoring completed
1,FFQ,w1ffq_avgkcal,SUM(w1ffq##_kcal) / 14,Average kcal consumed each day for the reported 14 days,NaN
2,FFQ,w1ffq##_gfat,w1Bx_ffq_## * fat_ffq##,Total fat grams consumed for each food variable,fat_ffq## comes from FFQ excel sheet; * assuming reverse scoring completed
3,FFQ,w1ffq##_fatkcal,w1ffq##_gfat*9,Total kcal derived from fat consumed for each food variable,NaN
4,FFQ,w1ffq_avgfatkcal,SUM(w1ffq##_fatkcal)/14,Average kcal from fat consumed each day for the reported 14 days,NaN


In [211]:
#display(reverse_scoring.head())

In [212]:
#s1_behav.columns.values

## Calculations
--- 

In [227]:
"""
# Methods for Behavioral Survey Calculations 
"""


"""
# FFQ : Food Frequency Questionnaire
"""

def ffq_calculations():
   
    # Set Variables

    ffq_dict={}
    nutrient_info_update={}
    s1_behav_copy=s1_behav    
    
    # loop through the ffq## values(columns)
    for col_val in s1_behav.filter(like="ffq", axis=1).columns.values:
                    
        # extract ffq numerical id
        id_val=col_val.split("_")[2]
        
        # create string with numerical id to search in nutrient document
        ffq_id="ffq%s"%id_val
        #print("\n[INFO] getting %s calculations"%(ffq_id))

        # prepare the column -- translte into numeric to run calculations
        s1_behav_copy[col_val]=pd.to_numeric(s1_behav_copy[col_val], errors='coerce')
        z=np.asarray(s1_behav_copy[col_val])
        #print("\n[INFO] original data: \n", wBx_ffq[:10])
        
        
        if ffq_id not in nutrient_info_update:
            nutrient_info_update[ffq_id]={}
            
            
        # -- Get nutrient ffq averages --
        # -- modify for portions as noted in dictionary key 
        
        # grams/med portion 
        grams_portion=float(nutrient_info.loc[ffq_id, 'grams/med portion '])
        
        # get kcal_ffq##
        kcal_ffq=(float(nutrient_info.loc[ffq_id, 'kcal/100g'])*grams_portion)/100
        
        # get fat_ffq##
        fat_ffq=(float(nutrient_info.loc[ffq_id, 'fat(g)/100g'])*grams_portion)/100

        # get sug_ffq
        sug_ffq=(float(nutrient_info.loc[ffq_id, 'sugars/100g'])*grams_portion)/100

        
        nutrient_info_update[ffq_id]['kcal/portion']=kcal_ffq
        nutrient_info_update[ffq_id]['fatg/portion']=fat_ffq
        nutrient_info_update[ffq_id]['sugarg/portion']=sug_ffq
        
        
        for bbx_id in s1_behav.index.values:
            
            if bbx_id not in ffq_dict:
                ffq_dict[bbx_id]={}
   
            # -- Apply reverse scoring to the ffq_## value --
            
            s1_behav_copy[col_val] = s1_behav_copy[col_val].replace([1,2,3,4,5,6],[0,2,5,8,11.5, 14])

            # get row
            ffq_val=s1_behav_copy.loc[bbx_id,col_val]
            
            # -- Calculations by ffq## --
            
            # -- ffq##_kcal -- 
            w1ffq_kcal=ffq_val*kcal_ffq
            #print("\n[INFO] %s * %s"%(col_val, kcal_ffq))
            #print("[INFO] kcal data: \n", w1ffq_kcal[:10]

            # -- ffq##_gfat -- 
            w1ffq_gfat=ffq_val*fat_ffq
            #print("\n[INFO] %s * %s"%(col_val, fat_ffq))
            #print("[INFO] gfat data: \n", w1ffq_gfat)

            # -- ffq##_fatkcal -- 
            w1ffq_fatkcal=w1ffq_gfat*9
            #print("[INFO] fatkcal data: ", w1ffq_fatkcal)

            # -- ffq##_gsug --
            w1ffq_gsug=ffq_val*sug_ffq
            #print("\n[INFO] %s * %s"%(col_val, sug_ffq))
            #print("[INFO] gsug data: \n", w1ffq_gsug)

            # -- ffq##_sugkcal --
            w1ffq_sugkcal = w1ffq_gsug*4
            #print("[INFO] sugkcal data: ", w1ffq_sugkcal)

            # -- Fill Dictionary --
            ffq_dict[bbx_id]['w1Bx_ffq_%s_kcal'%id_val]=w1ffq_kcal
            ffq_dict[bbx_id]['w1Bx_ffq_%s_gfat'%id_val]=w1ffq_gfat
            ffq_dict[bbx_id]['w1Bx_ffq_%s_fatkcal'%id_val]=w1ffq_fatkcal
            ffq_dict[bbx_id]['w1Bx_ffq_%s_gsug'%id_val]=w1ffq_gsug 
            ffq_dict[bbx_id]['w1Bx_ffq_%s_sugkcal'%id_val]=w1ffq_sugkcal 

    # Create dataframe from dicionary
    ffq_df= pd.DataFrame(ffq_dict).T  
    # Filter dataframe into specific sets for calculations
    kcal_df=ffq_df.filter(like="kcal", axis=1)
    fatkcal_df=ffq_df.filter(like="fatkcal", axis=1)
    gsug_df=ffq_df.filter(like="gsug", axis=1)
    sugkcal_df=ffq_df.filter(like="sugkcal", axis=1)

    for bbx_id in s1_behav.index.values:
        """
        # Set Rows
        """
        kcal_row=kcal_df.loc[bbx_id,:]
        fatkcal_row=fatkcal_df.loc[bbx_id,:]  
        sugkcal_row=sugkcal_df.loc[bbx_id, :]

        """
        # Calculations 
        """
        # --avgkcal --
        avgkcal=sum(kcal_row)/14

        # -- avgfatkcal -
        avgfatkcal = sum(fatkcal_row)/14

        # -- pctfat --
        pctfat=(sum(fatkcal_row)/sum(kcal_row))*100
        pctfat_d = (avgfatkcal/avgkcal)*100

        # -- avgsugkcal --
        avgsugkcal=sum(sugkcal_row)/14

        # -- pctsug --
        pctsug=(sum(sugkcal_row)/sum(kcal_row))*100
        pctsug_d=(avgsugkcal/avgkcal)*100

        """
        # Add to dictionary
        """
        # -- Fill Dictionary --
        ffq_dict[bbx_id]['w1ffq_avgkcal']=avgkcal
        ffq_dict[bbx_id]['w1ffq_avgfatkcal']=avgfatkcal
        ffq_dict[bbx_id]['w1ffq_avgsugkcal']=avgsugkcal
        ffq_dict[bbx_id]['w1ffq_pctfat']=pctfat
        ffq_dict[bbx_id]['w1ffq_pctfat_d']=pctfat_d
        ffq_dict[bbx_id]['w1ffq_pctsug']=pctsug
        ffq_dict[bbx_id]['w1ffq_pctsug_d']=pctsug_d
        
    """
    # Generate Output
    """
    
    # make dataframe 
    ffq_df_final= pd.DataFrame(ffq_dict).T  
    #display(ffq_df_final.head())
    
    reverse_data=s1_behav_copy.filter(like="ffq", axis=1)
    ffq_reverse_data=pd.DataFrame(reverse_data)
    #display(ffq_reverse_data.head())
    
    nutrient_info_updated=pd.DataFrame(nutrient_info_update)
    
    # wrote tp csv
    #path="C:\Users\19802\Documents\nibl\bbx_test_outputs"
    ffq_df_final.to_csv(os.path.join(output_path,"w1Bx_ffq_calculations.csv"))
    ffq_reverse_data.to_csv(os.path.join(output_path,"w1Bx_ffq_reverse_scores.csv"))
    
    ffq_summary_df=ffq_df_final[["w1ffq_avgkcal", "w1ffq_avgfatkcal",
                           "w1ffq_pctfat", "w1ffq_pctfat_d", "w1ffq_avgsugkcal", "w1ffq_pctsug", "w1ffq_pctsug_d" ]]
    
    ffq_summary_df.to_csv(os.path.join(output_path, 'w1Bx_ffq_summary_variables.csv'))
    
    print('\n[INFO] completed FFQ calculations.\n')
    return ffq_df_final, ffq_reverse_data, nutrient_info_updated;

    # ---
                        
"""
# BIS Calculation
"""
                        
def bis_calculation():
    # set starting variables
    bis_dict={}
    error_ids=[]
    s1_behav_copy=s1_behav        
    
    """
    # Reverse Scoring 
    """
    
    s1_behav_copy["w1Bx_bis_6"] = s1_behav_copy["w1Bx_bis_6"].replace([4,3,2,1], [1,2,3,4])    
    s1_behav_copy["w1Bx_bis_7"] = s1_behav_copy["w1Bx_bis_7"].replace([4,3,2,1], [1,2,3,4])
    s1_behav_copy["w1Bx_bis_8"] = s1_behav_copy["w1Bx_bis_8"].replace([4,3,2,1], [1,2,3,4])
    s1_behav_copy["w1Bx_bis_9"] = s1_behav_copy["w1Bx_bis_9"].replace([4,3,2,1], [1,2,3,4])
    s1_behav_copy["w1Bx_bis_10"] = s1_behav_copy["w1Bx_bis_10"].replace([4,3,2,1], [1,2,3,4])
    s1_behav_copy["w1Bx_bis_13"] = s1_behav_copy["w1Bx_bis_13"].replace([4,3,2,1], [1,2,3,4])

    
    for col_val in s1_behav.filter(regex="bis_[0-9]", axis=1).columns.values:
        bis_df=s1_behav_copy.filter(regex="bis_[0-9]", axis=1)
        bis_df=bis_df.apply(pd.to_numeric,  errors='coerce')
        id_val=col_val.split("_")[2]

        """
        # Calculations
        """


        for bbx_id in s1_behav.index.values:
            try:
                # -- bis_tot -- 
                bis_tot=sum(bis_df.loc[bbx_id,:])

                # -- bis_npi --
                bis_npi=sum(bis_df.loc[bbx_id,['w1Bx_bis_6', 'w1Bx_bis_7', 'w1Bx_bis_8', 'w1Bx_bis_9', 'w1Bx_bis_10']])

                # -- bis_mi -- 
                bis_mi=sum(bis_df.loc[bbx_id,['w1Bx_bis_1', 'w1Bx_bis_2', 'w1Bx_bis_3', 'w1Bx_bis_4', 'w1Bx_bis_5']])

                # -- bis_ai --
                bis_ai=sum(bis_df.loc[bbx_id,['w1Bx_bis_11', 'w1Bx_bis_12', 'w1Bx_bis_13', 'w1Bx_bis_14', 'w1Bx_bis_15']])

            except:
                error_ids.append(bbx_id)

            # set dictionary
            if bbx_id not in bis_dict:
                bis_dict[bbx_id]={}

            # add values to dictionary
            bis_dict[bbx_id]["w1bis_tot"]=bis_tot
            bis_dict[bbx_id]["w1bis_npi"]=bis_npi
            bis_dict[bbx_id]["w1bis_mi"]=bis_mi
            bis_dict[bbx_id]["w1bis_ai"]=bis_ai

    if not error_ids:
        pass
    else:
        print("\n[INFO] ERROR with BIS-15 calculation for ids: ",set(error_ids))
    
    # make dataframe 
    
    bis_df_final= pd.DataFrame(bis_dict).T  
    #display(bis_df_final.tail())
    
    bis_df_final.to_csv(os.path.join(output_path, "w1Bx_bis-15_calculations.csv"))
    print('\n[INFO] completed BIS-15 calculations.\n')
    
    
    return bis_df_final;


def fci_calculation():
    # set starting variables
    fci_dict={}
    error_ids=[]
    s1_behav_copy=s1_behav

    df=s1_behav_copy.filter(regex="fci", axis=1)
    df=df.apply(pd.to_numeric,  errors='coerce')
    #id_val=col_val.split("_")[2]
    fciCrv_df=df.filter(regex="fciCrv", axis=1)
    fciLik_df=df.filter(regex="fciLik", axis=1)
    
    
    """
    # Calculations
    """

    for bbx_id in s1_behav.index.values:
        try:
            # -- fci_crave --
            #fciCrv_sum=sum(fciCrv_df.loc[bbx_id, :])
            #fciCrv_ct=len(fciCrv_df.columns.values)
            #fci_crave=fciCrv_sum/fciCrv_ct
            fci_crave=fciCrv_df.loc[bbx_id, :].mean()

            # -- fci_chf --
            fci_chf=fciCrv_df.loc[bbx_id,['w1Bx_fciCrv_3' , 'w1Bx_fciCrv_6' , 'w1Bx_fciCrv_4' , 'w1Bx_fciCrv_15' ,
                                          'w1Bx_fciCrv_26' , 'w1Bx_fciCrv_19' , 'w1Bx_fciCrv_10' , 'w1Bx_fciCrv_27']].mean()

            # -- fci_cs -- 
            fci_cs=fciCrv_df.loc[bbx_id,['w1Bx_fciCrv_25' , 'w1Bx_fciCrv_16' , 'w1Bx_fciCrv_24' , 'w1Bx_fciCrv_17' , 'w1Bx_fciCrv_23' , 'w1Bx_fciCrv_1', 'w1Bx_fciCrv_8' , 'w1Bx_fciCrv_13']].mean()

            # -- fci_cs -- 
            fci_cc=fciCrv_df.loc[bbx_id,['w1Bx_fciCrv_21' , 'w1Bx_fciCrv_18' , 'w1Bx_fciCrv_12' , 'w1Bx_fciCrv_5' , 'w1Bx_fciCrv_9' , 'w1Bx_fciCrv_28' , 'w1Bx_fciCrv_14' , 'w1Bx_fciCrv_22']].mean()

            # -- fci_cs -- 
            fci_cff=fciCrv_df.loc[bbx_id,['w1Bx_fciCrv_11' , 'w1Bx_fciCrv_7' , 'w1Bx_fciCrv_20' , 'w1Bx_fciCrv_2']].mean()


            fci_like=fciLik_df.loc[bbx_id, :].mean()

            fci_lhf=fciLik_df.loc[bbx_id,['w1Bx_fciLik_3' , 'w1Bx_fciLik_6' , 'w1Bx_fciLik_4' , 'w1Bx_fciLik_15' , 'w1Bx_fciLik_26' , 'w1Bx_fciLik_19' , 'w1Bx_fciLik_10' , 'w1Bx_fciLik_27']].mean()

            fci_ls=fciLik_df.loc[bbx_id,['w1Bx_fciLik_25' , 'w1Bx_fciLik_16' , 'w1Bx_fciLik_24' , 'w1Bx_fciLik_17' , 'w1Bx_fciLik_23' , 'w1Bx_fciLik_1' , 'w1Bx_fciLik_8' , 'w1Bx_fciLik_13']].mean()

            fci_lc=fciLik_df.loc[bbx_id,['w1Bx_fciLik_21' , 'w1Bx_fciLik_18' , 'w1Bx_fciLik_12' , 'w1Bx_fciLik_5' , 'w1Bx_fciLik_9' , 'w1Bx_fciLik_28' , 'w1Bx_fciLik_14' , 'w1Bx_fciLik_22']].mean()

            fci_lff=fciLik_df.loc[bbx_id,['w1Bx_fciLik_11' , 'w1Bx_fciLik_7' , 'w1Bx_fciLik_20' , 'w1Bx_fciLik_2']].mean()

            
        except:
            error_ids.append(bbx_id)

        # set dictionary
        if bbx_id not in fci_dict:
            fci_dict[bbx_id]={}

        # add values to dictionary
        fci_dict[bbx_id]["w1fci_crave"]=fci_crave
        fci_dict[bbx_id]["w1fci_chf"]=fci_chf
        fci_dict[bbx_id]["w1fci_cs"]=fci_cs
        fci_dict[bbx_id]["w1fci_cc"]=fci_cc
        fci_dict[bbx_id]["w1fci_cff"]=fci_cff	
        fci_dict[bbx_id]["w1fci_like"]=fci_like
        fci_dict[bbx_id]["w1fci_lhf"]=fci_lhf
        fci_dict[bbx_id]["w1fci_ls"]=fci_ls
        fci_dict[bbx_id]["w1fci_lc"]=fci_lc	
        fci_dict[bbx_id]["w1fci_lff"]=fci_lff
        
        

    if not error_ids:
        pass
    else:
        print("\n[INFO] ERROR with FCI calculation for ids: ",set(error_ids))
    
    # make dataframe 
    
    df_final= pd.DataFrame(fci_dict).T  
    display(df_final.tail())
    
    df_final.to_csv(os.path.join(output_path, "w1Bx_fci_calculations.csv"))
    print('\n[INFO] completed FCI calculations.\n')
    
    
    return df_final;





def debq_calculation():
    # set starting variables
    debq_dict={}
    error_ids=[]
    s1_behav_copy=s1_behav        

    """
    # Reverse Scoring 
    """
    s1_behav_copy["w1Bx_debq_33"] = s1_behav_copy["w1Bx_debq_33"].replace(['5','4','3','2','1'], ['1','2','3','4','5'])  
    
    
    
    
    debq_df=s1_behav_copy.filter(like="debq", axis=1)
    debq_df=debq_df.apply(pd.to_numeric,  errors='coerce')
   
    #display(debq_df.head())
    
    """
    # Calculations
    """

    for bbx_id in s1_behav.index.values:
        try:

            debq_tot=debq_df.loc[bbx_id, :].mean()
            debq_r=debq_df.loc[bbx_id, ['w1Bx_debq_1' , 'w1Bx_debq_2' , 'w1Bx_debq_3' , 'w1Bx_debq_4' , 'w1Bx_debq_5' , 'w1Bx_debq_6' , 'w1Bx_debq_7' , 'w1Bx_debq_8' , 'w1Bx_debq_9' , 'w1Bx_debq_10']].mean()
            debq_e=debq_df.loc[bbx_id, ['w1Bx_debq_11' , 'w1Bx_debq_12' , 'w1Bx_debq_13' , 'w1Bx_debq_14' , 'w1Bx_debq_15' , 'w1Bx_debq_16' , 'w1Bx_debq_17',
                                        'w1Bx_debq_18' , 'w1Bx_debq_19' , 'w1Bx_debq_20' , 'w1Bx_debq_21' , 'w1Bx_debq_22' , 'w1Bx_debq_23']].mean()
            debq_ext=debq_df.loc[bbx_id, ['w1Bx_debq_24' , 'w1Bx_debq_25' , 'w1Bx_debq_26' , 'w1Bx_debq_27' , 'w1Bx_debq_28' , 'w1Bx_debq_29' , 'w1Bx_debq_30' , 'w1Bx_debq_31' , 'w1Bx_debq_32' , 'w1Bx_debq_33']].mean()



            # set dictionary
            if bbx_id not in debq_dict:
                debq_dict[bbx_id]={}

            # add values to dictionary
            debq_dict[bbx_id]["w1debq_tot"]=debq_tot
            debq_dict[bbx_id]["w1debq_r"]=debq_r
            debq_dict[bbx_id]["w1debq_e"]=debq_e
            debq_dict[bbx_id]["w1debq_ext"]=debq_ext
        except:
            error_ids.append(bbx_id)
        
    if not error_ids:
        pass
    else:
        print("\n[INFO] ERROR with DEBQ calculation for ids: ",set(error_ids))
    
    # make dataframe 
    
    df_final= pd.DataFrame(debq_dict).T  
    display(df_final.tail())
    
    df_final.to_csv(os.path.join(output_path, "w1Bx_debq_calculations.csv"))
    print('\n[INFO] completed DEBQ calculations.\n')
    
    
    return df_final;
        
    
    
def spsrq_calculation():
    # set starting variables
    spsrq_dict={}
    error_ids=[]
    s1_behav_copy=s1_behav        
    
    spsrq_df=s1_behav_copy.filter(like="spsrq", axis=1)
    spsrq_df=spsrq_df.apply(pd.to_numeric,  errors='coerce')
   
    #display(spsrq_df.head())
    
    """
    # Calculations
    """

    for bbx_id in s1_behav.index.values:
        try:
            spsrq_tot=sum(spsrq_df.loc[bbx_id, ['w1Bx_spsrq1' , 'w1Bx_spsrq2' , 'w1Bx_spsrq3' , 'w1Bx_spsrq4' , 'w1Bx_spsrq5' , 'w1Bx_spsrq6' , 'w1Bx_spsrq7' ,
                                                'w1Bx_spsrq8' , 'w1Bx_spsrq9' , 'w1Bx_spsrq10' , 'w1Bx_spsrq11' , 'w1Bx_spsrq12' , 'w1Bx_spsrq13' , 'w1Bx_spsrq14' , 'w1Bx_spsrq15' ,
                                                'w1Bx_spsrq16' , 'w1Bx_spsrq17' , 'w1Bx_spsrq18' , 'w1Bx_spsrq19' , 'w1Bx_spsrq20']])

            spsrq_sp=sum(spsrq_df.loc[bbx_id, ['w1Bx_spsrq1' , 'w1Bx_spsrq3' , 'w1Bx_spsrq5' , 'w1Bx_spsrq7' , 'w1Bx_spsrq9' , 'w1Bx_spsrq11' , 'w1Bx_spsrq13' , 'w1Bx_spsrq15' , 'w1Bx_spsrq17' , 'w1Bx_spsrq19']])

            spsrq_sr=sum(spsrq_df.loc[bbx_id, ['w1Bx_spsrq2' , 'w1Bx_spsrq4' , 'w1Bx_spsrq6' , 'w1Bx_spsrq8' , 'w1Bx_spsrq10' , 'w1Bx_spsrq12' , 'w1Bx_spsrq14' , 'w1Bx_spsrq16' , 'w1Bx_spsrq18' , 'w1Bx_spsrq20']])
        except:
            error_ids.append(bbx_id)
        # set dictionary
        if bbx_id not in spsrq_dict:
            spsrq_dict[bbx_id]={}

        # add values to dictionary
        spsrq_dict[bbx_id]["w1spsrq_tot"]=spsrq_tot
        spsrq_dict[bbx_id]["w1spsrq_sp"]=spsrq_sp
        spsrq_dict[bbx_id]["w1spsrq_sr"]=spsrq_sr
      
        
    if not error_ids:
        pass
    else:
        print("\n[INFO] ERROR with SPSRQ calculation for ids: ",set(error_ids))
    
    # make dataframe 
    
    df_final= pd.DataFrame(spsrq_dict).T  
    display(df_final.tail())
    
    df_final.to_csv(os.path.join(output_path, "w1Bx_spsrq_calculations.csv"))
    print('\n[INFO] completed SPSRQ calculations.\n')
    
    
    return df_final;
 
    
    
    

def bis_bas_calculation():
    # set starting variables
    bbas_dict={}
    error_ids=[]
    s1_behav_copy=s1_behav        
   
    #display(spsrq_df.head())
    """
    # Reverse Scoring 
    """
    
    #display(s1_behav_copy.filter(like="bis_bas", axis=1).head())
    
    cols=["w1Bx_bis_bas_1", 'w1Bx_bis_bas_3', 'w1Bx_bis_bas_4' , 'w1Bx_bis_bas_5', 'w1Bx_bis_bas_6', 'w1Bx_bis_bas_7', 'w1Bx_bis_bas_8', 
          'w1Bx_bis_bas_9', 'w1Bx_bis_bas_10', 'w1Bx_bis_bas_11', 'w1Bx_bis_bas_12', 'w1Bx_bis_bas_13', 'w1Bx_bis_bas_14', 'w1Bx_bis_bas_15', 'w1Bx_bis_bas_16', 
          'w1Bx_bis_bas_17', 'w1Bx_bis_bas_18', 'w1Bx_bis_bas_19', 'w1Bx_bis_bas_20', 'w1Bx_bis_bas_22', 'w1Bx_bis_bas_23']
    
    s1_behav_copy[cols]= s1_behav_copy[cols].replace(['4','3','2','1'], ['1','2','3','4'])
    
    #display(s1_behav_copy.filter(like="bis_bas", axis=1).head())
    
    bbas_df=s1_behav_copy.filter(like="bis_bas", axis=1)
    bbas_df=bbas_df.apply(pd.to_numeric,  errors='coerce')

    """
    # Calculations
    """
    
    for bbx_id in s1_behav.index.values:
        try:
            bisbas_bastot=sum(bbas_df.loc[bbx_id, ['w1Bx_bis_bas_3' , 'w1Bx_bis_bas_8' , 'w1Bx_bis_bas_11' , 'w1Bx_bis_bas_20',
                                                    'w1Bx_bis_bas_5' , 'w1Bx_bis_bas_9' , 'w1Bx_bis_bas_14' , 'w1Bx_bis_bas_19', 'w1Bx_bis_bas_4',
                                                    'w1Bx_bis_bas_6' , 'w1Bx_bis_bas_13' , 'w1Bx_bis_bas_17' , 'w1Bx_bis_bas_22']])

            bisbas_basd=sum(bbas_df.loc[bbx_id, ['w1Bx_bis_bas_3' , 'w1Bx_bis_bas_8' , 'w1Bx_bis_bas_11' , 'w1Bx_bis_bas_20']])

            bisbas_basfs=sum(bbas_df.loc[bbx_id, ['w1Bx_bis_bas_5' , 'w1Bx_bis_bas_9' , 'w1Bx_bis_bas_14' , 'w1Bx_bis_bas_19']])

            bisbas_basrr=sum(bbas_df.loc[bbx_id, ['w1Bx_bis_bas_4' , 'w1Bx_bis_bas_6' , 'w1Bx_bis_bas_13' , 'w1Bx_bis_bas_17' , 'w1Bx_bis_bas_22']])

            bisbas_bis=sum(bbas_df.loc[bbx_id, ['w1Bx_bis_bas_2' , 'w1Bx_bis_bas_7' , 'w1Bx_bis_bas_12' , 'w1Bx_bis_bas_15' , 'w1Bx_bis_bas_18' , 'w1Bx_bis_bas_21' , 'w1Bx_bis_bas_23']])
        except:
            error_ids.append(bbx_id)
        # set dictionary
        if bbx_id not in bbas_dict:
            bbas_dict[bbx_id]={}

        # add values to dictionary
        bbas_dict[bbx_id]["w1bisbas_bastot"]=bisbas_bastot
        bbas_dict[bbx_id]["w1bisbas_basd"]=bisbas_basd
        bbas_dict[bbx_id]["w1bisbas_basfs"]=bisbas_basfs
        bbas_dict[bbx_id]["w1bisbas_basrr"]=bisbas_basrr
        bbas_dict[bbx_id]["w1bisbas_bis"]=bisbas_bis
        
    if not error_ids:
        pass
    else:
        print("\n[INFO] ERROR with BIS/BAS calculation for ids: ",set(error_ids))
    
    # make dataframe 
    
    df_final= pd.DataFrame(bbas_dict).T  
    display(df_final.tail())
    
    df_final.to_csv(os.path.join(output_path, "w1Bx_bis_bas_calculations.csv"))
    print('\n[INFO] completed BIS/BAS calculations.\n')
    
    
    return df_final;




def pfs_calculation():
    # set starting variables
    pfs_dict={}
    error_ids=[]
    s1_behav_copy=s1_behav        
        
    pfs_df=s1_behav_copy.filter(like="pfs", axis=1)
    pfs_df=pfs_df.apply(pd.to_numeric,  errors='coerce')

    """
    # Calculations
    """
    
    for bbx_id in s1_behav.index.values:
        try:
            pfs_tot=pfs_df.loc[bbx_id, ['w1Bx_pfs_1' , 'w1Bx_pfs_3' , 'w1Bx_pfs_8' , 'w1Bx_pfs_16' , 'w1Bx_pfs_17' , 'w1Bx_pfs_19', 'w1Bx_pfs_5' , 'w1Bx_pfs_6' , 
                                                 'w1Bx_pfs_10' , 'w1Bx_pfs_11', 'w1Bx_pfs_14' , 'w1Bx_pfs_15' , 'w1Bx_pfs_18' , 'w1Bx_pfs_20' , 'w1Bx_pfs_21']].mean()

            pfs_fa=pfs_df.loc[bbx_id, ['w1Bx_pfs_1' , 'w1Bx_pfs_3' , 'w1Bx_pfs_8' , 'w1Bx_pfs_16' , 'w1Bx_pfs_17' , 'w1Bx_pfs_19']].mean()
            pfs_fp=pfs_df.loc[bbx_id, ['w1Bx_pfs_5' , 'w1Bx_pfs_6' , 'w1Bx_pfs_10' , 'w1Bx_pfs_11']].mean()
            pfs_ft=pfs_df.loc[bbx_id, ['w1Bx_pfs_14' , 'w1Bx_pfs_15' , 'w1Bx_pfs_18' , 'w1Bx_pfs_20' , 'w1Bx_pfs_21']].mean()

        except:
            error_ids.append(bbx_id)
            
        # set dictionary
        if bbx_id not in pfs_dict:
            pfs_dict[bbx_id]={}

        # add values to dictionary
        pfs_dict[bbx_id]["w1pfs_tot"]=pfs_tot
        pfs_dict[bbx_id]["w1pfs_fa"]=pfs_fa
        pfs_dict[bbx_id]["w1pfs_fp"]=pfs_fp
        pfs_dict[bbx_id]["w1pfs_ft"]=pfs_ft
        
    if not error_ids:
        pass
    else:
        print("\n[INFO] ERROR with PFS calculation for ids: ",set(error_ids))
    
    # make dataframe 
    
    df_final= pd.DataFrame(pfs_dict).T  
    display(df_final.tail())
    
    #df_final.to_csv(os.path.join(output_path, "w1Bx_pfs_calculations.csv"))
    print('\n[INFO] completed PFS calculations.\n')
    
    
    return df_final;
        
    
    
    
def yfas_calculation():
    # set starting variables
    yfas_dict={}
    error_ids=[]
    s1_behav_copy=s1_behav        
    sess_id="w1"
    #display(spsrq_df.head())
    """
    # Reverse Scoring 
    """
    
    #display(s1_behav_copy.filter(like="yfas", axis=1).head())
    cols1=['w1Bx_yfas_1' , 'w1Bx_yfas_2' , 'w1Bx_yfas_4' , 'w1Bx_yfas_6', 'w1Bx_yfas_3',
           'w1Bx_yfas_5', 'w1Bx_yfas_7' , 'w1Bx_yfas_9' , 'w1Bx_yfas_12' , 
           'w1Bx_yfas_13', 'w1Bx_yfas_14', 'w1Bx_yfas_15', 'w1Bx_yfas_16']
    s1_behav_copy[cols1]= s1_behav_copy[cols1].replace(['1','2','3','4'], ['0','0','0','1'])
    
    cols2=[ 'w1Bx_yfas_8', 'w1Bx_yfas_10', 'w1Bx_yfas_11']
    s1_behav_copy[cols2]= s1_behav_copy[cols2].replace(['1','2','3','4'], ['0','0','1','1'])
    
    cols3=['w1Bx_yfas_17_3', 'w1Bx_yfas_17_4', 'w1Bx_yfas_17_5', 'w1Bx_yfas_17_6']
    s1_behav_copy[cols3]= s1_behav_copy[cols3].replace(['1','2'], ['0','1'])
    
    s1_behav_copy['w1Bx_yfas_17_8']= s1_behav_copy['w1Bx_yfas_17_8'].replace(['2'], ['0'])
    
    s1_behav_copy['w1Bx_yfas_25']= s1_behav_copy['w1Bx_yfas_25'].replace(['1','2', '3', '4', '5'], ['0', '0', '0', '0', '1'])
    
    #display(s1_behav_copy.filter(like="yfas", axis=1).head())

    
    yfas_df=s1_behav_copy.filter(like="yfas", axis=1)
    yfas_df=yfas_df.apply(pd.to_numeric,  errors='coerce')

    #display(yfas_df.head())
    
    """
    # Calculations
    """
    
    for bbx_id in s1_behav.index.values:
        try:
            yfas_c1=sum(yfas_df.loc[bbx_id, ['%sBx_yfas_1'%sess_id , '%sBx_yfas_2'%sess_id , '%sBx_yfas_3'%sess_id]])

            yfas_c2=sum(yfas_df.loc[bbx_id, ['%sBx_yfas_4'%sess_id , '%sBx_yfas_17_6'%sess_id , '%sBx_yfas_17_8'%sess_id,
                                            '%sBx_yfas_25'%sess_id]])
            yfas_c3=sum(yfas_df.loc[bbx_id, ['%sBx_yfas_5'%sess_id , '%sBx_yfas_6'%sess_id , '%sBx_yfas_7'%sess_id]])

            yfas_c4=sum(yfas_df.loc[bbx_id, ['%sBx_yfas_8'%sess_id , '%sBx_yfas_9'%sess_id , '%sBx_yfas_10'%sess_id,
                                            '%sBx_yfas_11'%sess_id ]])

            yfas_c5=float(yfas_df.loc[bbx_id, ['%sBx_yfas_17_3'%sess_id ]])

            yfas_c6=sum(yfas_df.loc[bbx_id, ['%sBx_yfas_17_4'%sess_id, '%sBx_yfas_17_5'%sess_id]])

            yfas_c7=sum(yfas_df.loc[bbx_id, ['%sBx_yfas_12'%sess_id, '%sBx_yfas_13'%sess_id, '%sBx_yfas_14'%sess_id ]])

            yfas_c8=sum(yfas_df.loc[bbx_id, ['%sBx_yfas_15'%sess_id, '%sBx_yfas_16'%sess_id]])

            #print(yfas_df.loc[bbx_id, ['%sBx_yfas_15'%sess_id, '%sBx_yfas_16'%sess_id]])
            #print(yfas_c1, yfas_c2, yfas_c3, yfas_c4, yfas_c5,yfas_c6,yfas_c7,yfas_c8)

            if yfas_c1 == 0:
                yfas_c1_met=0
            elif yfas_c1 >=1 :
                yfas_c1_met=1
            else:
                pass

            if yfas_c2 == 0:
                yfas_c2_met=0
            elif yfas_c2 >= 1:
                yfas_c2_met=1

            if yfas_c3 == 0:
                yfas_c3_met=0
            elif yfas_c3 >= 1:
                yfas_c3_met=1

            if yfas_c4 == 0:
                yfas_c4_met=0
            elif yfas_c4 >= 1:
                yfas_c4_met=1

            if yfas_c5 == 0:
                yfas_c5_met=0
            elif yfas_c5 >= 1:
                yfas_c5_met=1

            if yfas_c6 == 0:
                yfas_c6_met=0
            elif yfas_c26 >= 1:
                yfas_c6_met=1


            if yfas_c7 == 0:
                yfas_c7_met=0
            elif yfas_c7 >= 1:
                yfas_c7_met=1

            if yfas_c8 == 0:
                yfas_c8_met=0
            elif 8 >= 1:
                yfas_c8=met=1

            yfas_cont_score=yfas_c1_met+yfas_c2_met+yfas_c3_met+yfas_c4_met+yfas_c5_met+yfas_c6_met+yfas_c7_met+yfas_c8_met
            #print(yfas_cont_score)

        except:
            error_ids.append(bbx_id)
            
        # set dictionary
        if bbx_id not in yfas_dict:
            yfas_dict[bbx_id]={}

        # add values to dictionary
        yfas_dict[bbx_id]["%syfas_c1"%sess_id]=yfas_c1
        yfas_dict[bbx_id]["%syfas_c2"%sess_id]=yfas_c2
        yfas_dict[bbx_id]["%syfas_c3"%sess_id]=yfas_c3
        yfas_dict[bbx_id]["%syfas_c4"%sess_id]=yfas_c4
        yfas_dict[bbx_id]["%syfas_c5"%sess_id]=yfas_c5
        yfas_dict[bbx_id]["%syfas_c6"%sess_id]=yfas_c6
        yfas_dict[bbx_id]["%syfas_c7"%sess_id]=yfas_c7
        yfas_dict[bbx_id]["%syfas_c8"%sess_id]=yfas_c8
        yfas_dict[bbx_id]["%syfas_c1_met"%sess_id]=yfas_c1_met
        yfas_dict[bbx_id]["%syfas_c2_met"%sess_id]=yfas_c2_met
        yfas_dict[bbx_id]["%syfas_c3_met"%sess_id]=yfas_c3_met
        yfas_dict[bbx_id]["%syfas_c4_met"%sess_id]=yfas_c4_met
        yfas_dict[bbx_id]["%syfas_c5_met"%sess_id]=yfas_c5_met
        yfas_dict[bbx_id]["%syfas_c6_met"%sess_id]=yfas_c6_met
        yfas_dict[bbx_id]["%syfas_c7_met"%sess_id]=yfas_c7_met
        yfas_dict[bbx_id]["%syfas_c8_met"%sess_id]=yfas_c8_met
        yfas_dict[bbx_id]["%syfas_cont_score"%sess_id]=yfas_cont_score
        
        
    if not error_ids:
        pass
    else:
        print("\n[INFO] ERROR with YFAS calculation for ids: ",set(error_ids))
    
    # make dataframe 
    
    df_final= pd.DataFrame(yfas_dict).T  
    display(df_final.tail(20))
    
    df_final.to_csv(os.path.join(output_path, "w1Bx_yfas_calculations.csv"))
    print('\n[INFO] completed YFAS calculations.\n')
 

  
## Food Frequency Questionnaire (FFQ)   <a anchor="class" id="ffq"></a>
---


View FFQ Calculations

Get Calculations

In [228]:
# Get Calculations
ffq_summary_vars, ffq_reverse_data, nutrient_info_updated=ffq_calculations()


[INFO] completed FFQ calculations.



### Quality Check  
---

Check for null and other error values

In [229]:
# add column with count of null values found in the subject
ffq_reverse_data["null_ct"]=ffq_reverse_data.isnull().sum(axis=1)
ffq_summary_vars["null_ct"]=ffq_summary_vars.isnull().sum(axis=1)

In [230]:
ffq_reverse_data.head()

,w1Bx_ffq_1,w1Bx_ffq_2,w1Bx_ffq_3,w1Bx_ffq_4,w1Bx_ffq_5,w1Bx_ffq_6,w1Bx_ffq_7,w1Bx_ffq_8,w1Bx_ffq_9,w1Bx_ffq_10,w1Bx_ffq_11,w1Bx_ffq_12,w1Bx_ffq_13,w1Bx_ffq_14,w1Bx_ffq_15,w1Bx_ffq_16,w1Bx_ffq_17,w1Bx_ffq_18,w1Bx_ffq_19,w1Bx_ffq_20,w1Bx_ffq_21,w1Bx_ffq_22,w1Bx_ffq_23,w1Bx_ffq_24,w1Bx_ffq_25,w1Bx_ffq_26,w1Bx_ffq_27,w1Bx_ffq_28,w1Bx_ffq_29,w1Bx_ffq_30,w1Bx_ffq_31,w1Bx_ffq_32,w1Bx_ffq_33,w1Bx_ffq_34,w1Bx_ffq_35,w1Bx_ffq_36,w1Bx_ffq_37,w1Bx_ffq_38,w1Bx_ffq_39,w1Bx_ffq_40,w1Bx_ffq_41,w1Bx_ffq_42,w1Bx_ffq_43,w1Bx_ffq_44,w1Bx_ffq_45,w1Bx_ffq_46,w1Bx_ffq_47,w1Bx_ffq_48,w1Bx_ffq_49,w1Bx_ffq_50,w1Bx_ffq_51,w1Bx_ffq_52,w1Bx_ffq_53,w1Bx_ffq_54,w1Bx_ffq_55,w1Bx_ffq_56,w1Bx_ffq_57,w1Bx_ffq_58,w1Bx_ffq_59,w1Bx_ffq_60,w1Bx_ffq_61,w1Bx_ffq_62,w1Bx_ffq_63,w1Bx_ffq_64,w1Bx_ffq_65,w1Bx_ffq_66,w1Bx_ffq_67,w1Bx_ffq_68,w1Bx_ffq_69,w1Bx_ffq_70,w1Bx_ffq_71,w1Bx_ffq_72,w1Bx_ffq_73,w1Bx_ffq_74,w1Bx_ffq_75,w1Bx_ffq_76,null_ct
patID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
bbx_001,11.5,11.5,0.0,0.0,11.5,11.5,0.0,11.5,8.0,11.5,2.0,11.5,11.5,0.0,0.0,11.5,2.0,0.0,2.0,0.0,0.0,2.0,2.0,0.0,2.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,2.0,2.0,0.0,2.0,2.0,0.0,2.0,0.0,2.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,2.0,8.0,2.0,2.0,0.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,11.5,0.0,0.0,8.0,0.0,0
bbx_002,11.5,14.0,2.0,0.0,0.0,8.0,8.0,2.0,11.5,11.5,2.0,11.5,11.5,0.0,2.0,2.0,2.0,0.0,0.0,0.0,2.0,11.5,2.0,0.0,0.0,2.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,0.0,2.0,0.0,0.0,0.0,2.0,0.0,11.5,0.0,0.0,0.0,11.5,2.0,0.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,8.0,0.0,0.0,0.0,0
bbx_003,14.0,14.0,11.5,0.0,11.5,8.0,0.0,11.5,2.0,11.5,0.0,11.5,2.0,0.0,11.5,8.0,11.5,0.0,0.0,0.0,2.0,8.0,0.0,0.0,8.0,2.0,0.0,8.0,0.0,2.0,2.0,0.0,0.0,2.0,0.0,2.0,0.0,11.5,11.5,0.0,11.5,11.5,8.0,0.0,14.0,11.5,11.5,11.5,2.0,11.5,11.5,2.0,11.5,2.0,2.0,0.0,14.0,11.5,0.0,0.0,0.0,0.0,11.5,0.0,11.5,0.0,2.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,11.5,0.0,0
bbx_004,2.0,2.0,2.0,0.0,11.5,8.0,8.0,2.0,2.0,11.5,11.5,2.0,2.0,2.0,0.0,2.0,2.0,0.0,0.0,0.0,0.0,11.5,2.0,0.0,0.0,2.0,2.0,2.0,2.0,2.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,11.5,0.0,0.0,0.0,0.0,2.0,11.5,2.0,0.0,2.0,0.0,0.0,2.0,0.0,11.5,0.0,11.5,2.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,11.5,11.5,0.0,0.0,0.0,0.0,0.0,0.0,8.0,14.0,0.0,0.0,2.0,0
bbx_005,11.5,2.0,0.0,0.0,2.0,2.0,11.5,0.0,2.0,2.0,0.0,2.0,11.5,2.0,2.0,11.5,11.5,2.0,0.0,0.0,11.5,11.5,2.0,0.0,2.0,0.0,0.0,2.0,0.0,2.0,0.0,2.0,2.0,2.0,0.0,0.0,0.0,11.5,2.0,0.0,11.5,2.0,0.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0,2.0,0.0,2.0,0.0,8.0,2.0,11.5,0.0,0.0,0.0,0.0,0.0,11.5,0.0,8.0,0.0,2.0,0.0,11.5,0.0,0.0,8.0,8.0,2.0,0.0,0.0,0


In [231]:
ffq_summary_vars.head()

,w1Bx_ffq_1_kcal,w1Bx_ffq_1_gfat,w1Bx_ffq_1_fatkcal,w1Bx_ffq_1_gsug,w1Bx_ffq_1_sugkcal,w1Bx_ffq_2_kcal,w1Bx_ffq_2_gfat,w1Bx_ffq_2_fatkcal,w1Bx_ffq_2_gsug,w1Bx_ffq_2_sugkcal,w1Bx_ffq_3_kcal,w1Bx_ffq_3_gfat,w1Bx_ffq_3_fatkcal,w1Bx_ffq_3_gsug,w1Bx_ffq_3_sugkcal,w1Bx_ffq_4_kcal,w1Bx_ffq_4_gfat,w1Bx_ffq_4_fatkcal,w1Bx_ffq_4_gsug,w1Bx_ffq_4_sugkcal,w1Bx_ffq_5_kcal,w1Bx_ffq_5_gfat,w1Bx_ffq_5_fatkcal,w1Bx_ffq_5_gsug,w1Bx_ffq_5_sugkcal,w1Bx_ffq_6_kcal,w1Bx_ffq_6_gfat,w1Bx_ffq_6_fatkcal,w1Bx_ffq_6_gsug,w1Bx_ffq_6_sugkcal,w1Bx_ffq_7_kcal,w1Bx_ffq_7_gfat,w1Bx_ffq_7_fatkcal,w1Bx_ffq_7_gsug,w1Bx_ffq_7_sugkcal,w1Bx_ffq_8_kcal,w1Bx_ffq_8_gfat,w1Bx_ffq_8_fatkcal,w1Bx_ffq_8_gsug,w1Bx_ffq_8_sugkcal,w1Bx_ffq_9_kcal,w1Bx_ffq_9_gfat,w1Bx_ffq_9_fatkcal,w1Bx_ffq_9_gsug,w1Bx_ffq_9_sugkcal,w1Bx_ffq_10_kcal,w1Bx_ffq_10_gfat,w1Bx_ffq_10_fatkcal,w1Bx_ffq_10_gsug,w1Bx_ffq_10_sugkcal,w1Bx_ffq_11_kcal,w1Bx_ffq_11_gfat,w1Bx_ffq_11_fatkcal,w1Bx_ffq_11_gsug,w1Bx_ffq_11_sugkcal,w1Bx_ffq_12_kcal,w1Bx_ffq_12_gfat,w1Bx_ffq_12_fatkcal,w1Bx_ffq_12_gsug,w1Bx_ffq_12_sugkcal,w1Bx_ffq_13_kcal,w1Bx_ffq_13_gfat,w1Bx_ffq_13_fatkcal,w1Bx_ffq_13_gsug,w1Bx_ffq_13_sugkcal,w1Bx_ffq_14_kcal,w1Bx_ffq_14_gfat,w1Bx_ffq_14_fatkcal,w1Bx_ffq_14_gsug,w1Bx_ffq_14_sugkcal,w1Bx_ffq_15_kcal,w1Bx_ffq_15_gfat,w1Bx_ffq_15_fatkcal,w1Bx_ffq_15_gsug,w1Bx_ffq_15_sugkcal,w1Bx_ffq_16_kcal,w1Bx_ffq_16_gfat,w1Bx_ffq_16_fatkcal,w1Bx_ffq_16_gsug,w1Bx_ffq_16_sugkcal,w1Bx_ffq_17_kcal,w1Bx_ffq_17_gfat,w1Bx_ffq_17_fatkcal,w1Bx_ffq_17_gsug,w1Bx_ffq_17_sugkcal,w1Bx_ffq_18_kcal,w1Bx_ffq_18_gfat,w1Bx_ffq_18_fatkcal,w1Bx_ffq_18_gsug,w1Bx_ffq_18_sugkcal,w1Bx_ffq_19_kcal,w1Bx_ffq_19_gfat,w1Bx_ffq_19_fatkcal,w1Bx_ffq_19_gsug,w1Bx_ffq_19_sugkcal,w1Bx_ffq_20_kcal,w1Bx_ffq_20_gfat,w1Bx_ffq_20_fatkcal,w1Bx_ffq_20_gsug,w1Bx_ffq_20_sugkcal,w1Bx_ffq_21_kcal,w1Bx_ffq_21_gfat,w1Bx_ffq_21_fatkcal,w1Bx_ffq_21_gsug,w1Bx_ffq_21_sugkcal,w1Bx_ffq_22_kcal,w1Bx_ffq_22_gfat,w1Bx_ffq_22_fatkcal,w1Bx_ffq_22_gsug,w1Bx_ffq_22_sugkcal,w1Bx_ffq_23_kcal,w1Bx_ffq_23_gfat,w1Bx_ffq_23_fatkcal,w1Bx_ffq_23_gsug,w1Bx_ffq_23_sugkcal,w1Bx_ffq_24_kcal,w1Bx_ffq_24_gfat,w1Bx_ffq_24_fatkcal,w1Bx_ffq_24_gsug,w1Bx_ffq_24_sugkcal,w1Bx_ffq_25_kcal,w1Bx_ffq_25_gfat,w1Bx_ffq_25_fatkcal,w1Bx_ffq_25_gsug,w1Bx_ffq_25_sugkcal,w1Bx_ffq_26_kcal,w1Bx_ffq_26_gfat,w1Bx_ffq_26_fatkcal,w1Bx_ffq_26_gsug,w1Bx_ffq_26_sugkcal,w1Bx_ffq_27_kcal,w1Bx_ffq_27_gfat,w1Bx_ffq_27_fatkcal,w1Bx_ffq_27_gsug,w1Bx_ffq_27_sugkcal,w1Bx_ffq_28_kcal,w1Bx_ffq_28_gfat,w1Bx_ffq_28_fatkcal,w1Bx_ffq_28_gsug,w1Bx_ffq_28_sugkcal,w1Bx_ffq_29_kcal,w1Bx_ffq_29_gfat,w1Bx_ffq_29_fatkcal,w1Bx_ffq_29_gsug,w1Bx_ffq_29_sugkcal,w1Bx_ffq_30_kcal,w1Bx_ffq_30_gfat,w1Bx_ffq_30_fatkcal,w1Bx_ffq_30_gsug,w1Bx_ffq_30_sugkcal,w1Bx_ffq_31_kcal,w1Bx_ffq_31_gfat,w1Bx_ffq_31_fatkcal,w1Bx_ffq_31_gsug,w1Bx_ffq_31_sugkcal,w1Bx_ffq_32_kcal,w1Bx_ffq_32_gfat,w1Bx_ffq_32_fatkcal,w1Bx_ffq_32_gsug,w1Bx_ffq_32_sugkcal,w1Bx_ffq_33_kcal,w1Bx_ffq_33_gfat,w1Bx_ffq_33_fatkcal,w1Bx_ffq_33_gsug,w1Bx_ffq_33_sugkcal,w1Bx_ffq_34_kcal,w1Bx_ffq_34_gfat,w1Bx_ffq_34_fatkcal,w1Bx_ffq_34_gsug,w1Bx_ffq_34_sugkcal,w1Bx_ffq_35_kcal,w1Bx_ffq_35_gfat,w1Bx_ffq_35_fatkcal,w1Bx_ffq_35_gsug,w1Bx_ffq_35_sugkcal,w1Bx_ffq_36_kcal,w1Bx_ffq_36_gfat,w1Bx_ffq_36_fatkcal,w1Bx_ffq_36_gsug,w1Bx_ffq_36_sugkcal,w1Bx_ffq_37_kcal,w1Bx_ffq_37_gfat,w1Bx_ffq_37_fatkcal,w1Bx_ffq_37_gsug,w1Bx_ffq_37_sugkcal,w1Bx_ffq_38_kcal,w1Bx_ffq_38_gfat,w1Bx_ffq_38_fatkcal,w1Bx_ffq_38_gsug,w1Bx_ffq_38_sugkcal,w1Bx_ffq_39_kcal,w1Bx_ffq_39_gfat,w1Bx_ffq_39_fatkcal,w1Bx_ffq_39_gsug,w1Bx_ffq_39_sugkcal,w1Bx_ffq_40_kcal,w1Bx_ffq_40_gfat,w1Bx_ffq_40_fatkcal,w1Bx_ffq_40_gsug,w1Bx_ffq_40_sugkcal,w1Bx_ffq_41_kcal,w1Bx_ffq_41_gfat,w1Bx_ffq_41_fatkcal,w1Bx_ffq_41_gsug,w1Bx_ffq_41_sugkcal,w1Bx_ffq_42_kcal,w1Bx_ffq_42_gfat,w1Bx_ffq_42_fatkcal,w1Bx_ffq_42_gsug,w1Bx_ffq_42_sugkcal,w1Bx_ffq_43_kcal,w1Bx_ffq_43_gfat,w1Bx_ffq_43_fatkcal,w1Bx_ffq_43_gsug,w1Bx_ffq_43_sugkcal,w1Bx_ffq_44_kcal,w1Bx_ffq_44_gfat,w1Bx_ffq_44_fatkcal,w1Bx_ffq_44_gsug,w1Bx_ffq_44_sugkcal,w1Bx_ffq_45_kcal,w1Bx_ffq_45_gfat,w1Bx_f

In [232]:
"""
# How to verify calculations?
"""

# load input data of testing data (expected scores)
ffq_data_test=pd.read_excel(os.path.join(data_path, "bbx_inputs", "ffq_test_scoring.xlsx"), sheet_name="data")
#ffq_data_test.index=ffq_data_test["patID"] # format dataframe

ffq_kcal_test=pd.read_excel(os.path.join(data_path, "bbx_inputs", "ffq_test_scoring.xlsx"), sheet_name="kcal")
#ffq_kcal_test.index=ffq_kcal_test["patID"] # format dataframe

ffq_fatkcal_test=pd.read_excel(os.path.join(data_path, "bbx_inputs", "ffq_test_scoring.xlsx"), sheet_name="fatkcal")
#ffq_fatkcal_test.index=ffq_fatkcal_test["patID"] # format dataframe

ffq_sugkcal_test=pd.read_excel(os.path.join(data_path, "bbx_inputs", "ffq_test_scoring.xlsx"), sheet_name="sugkcal")
#ffq_sugkcal_test.index=ffq_sugkcal_test["patID"] # format dataframe



Look at Reverse Scoring

Test data

In [233]:
display(ffq_data_test)

,patID,w1Bx_ffq_1,w1Bx_ffq_2,w1Bx_ffq_3,w1Bx_ffq_4,w1Bx_ffq_5,w1Bx_ffq_6,w1Bx_ffq_7,w1Bx_ffq_8,w1Bx_ffq_9,w1Bx_ffq_10,w1Bx_ffq_11,w1Bx_ffq_12,w1Bx_ffq_13,w1Bx_ffq_14,w1Bx_ffq_15,w1Bx_ffq_16,w1Bx_ffq_17,w1Bx_ffq_18,w1Bx_ffq_19,w1Bx_ffq_20,w1Bx_ffq_21,w1Bx_ffq_22,w1Bx_ffq_23,w1Bx_ffq_24,w1Bx_ffq_25,w1Bx_ffq_26,w1Bx_ffq_27,w1Bx_ffq_28,w1Bx_ffq_29,w1Bx_ffq_30,w1Bx_ffq_31,w1Bx_ffq_32,w1Bx_ffq_33,w1Bx_ffq_34,w1Bx_ffq_35,w1Bx_ffq_36,w1Bx_ffq_37,w1Bx_ffq_38,w1Bx_ffq_39,w1Bx_ffq_40,w1Bx_ffq_41,w1Bx_ffq_42,w1Bx_ffq_43,w1Bx_ffq_44,w1Bx_ffq_45,w1Bx_ffq_46,w1Bx_ffq_47,w1Bx_ffq_48,w1Bx_ffq_49,w1Bx_ffq_50,w1Bx_ffq_51,w1Bx_ffq_52,w1Bx_ffq_53,w1Bx_ffq_54,w1Bx_ffq_55,w1Bx_ffq_56,w1Bx_ffq_57,w1Bx_ffq_58,w1Bx_ffq_59,w1Bx_ffq_60,w1Bx_ffq_61,w1Bx_ffq_62,w1Bx_ffq_63,w1Bx_ffq_64,w1Bx_ffq_65,w1Bx_ffq_66,w1Bx_ffq_67,w1Bx_ffq_68,w1Bx_ffq_69,w1Bx_ffq_70,w1Bx_ffq_71,w1Bx_ffq_72,w1Bx_ffq_73,w1Bx_ffq_74,w1Bx_ffq_75,w1Bx_ffq_76
0,bbx_001,11.5,11.5,0.0,0,11.5,11.5,0.0,11.5,8.0,11.5,2.0,11.5,11.5,0,0.0,11.5,2.0,0,2,0,0.0,2.0,2,0,2,0,0,2,0,0,0,0,2,2,0,2,2,0.0,2.0,0,2.0,0.0,0,0,0.0,2.0,0.0,0.0,0,2.0,8.0,2,2.0,0,2.0,2,0.0,0.0,0,0,0,0,0.0,2.0,2.0,0,0,0,0.0,0,0,11.5,0,0,8.0,0
1,bbx_002,11.5,14.0,2.0,0,0.0,8.0,8.0,2.0,11.5,11.5,2.0,11.5,11.5,0,2.0,2.0,2.0,0,0,0,2.0,11.5,2,0,0,2,0,0,0,2,0,0,0,0,0,0,0,2.0,2.0,0,2.0,0.0,0,0,2.0,0.0,11.5,0.0,0,0.0,11.5,2,0.0,2,2.0,2,0.0,0.0,0,0,0,0,2.0,2.0,0.0,0,0,0,0.0,2,0,0.0,8,0,0.0,0
2,bbx_003,14.0,14.0,11.5,0,11.5,8.0,0.0,11.5,2.0,11.5,0.0,11.5,2.0,0,11.5,8.0,11.5,0,0,0,2.0,8.0,0,0,8,2,0,8,0,2,2,0,0,2,0,2,0,11.5,11.5,0,11.5,11.5,8,0,14.0,11.5,11.5,11.5,2,11.5,11.5,2,11.5,2,2.0,0,14.0,11.5,0,0,0,0,11.5,0.0,11.5,0,2,0,0.0,0,0,2.0,0,0,11.5,0
3,bbx_004,2.0,2.0,2.0,0,11.5,8.0,8.0,2.0,2.0,11.5,11.5,2.0,2.0,2,0.0,2.0,2.0,0,0,0,0.0,11.5,2,0,0,2,2,2,2,2,0,0,2,0,0,0,0,0.0,11.5,0,0.0,0.0,0,2,11.5,2.0,0.0,2.0,0,0.0,2.0,0,11.5,0,11.5,2,0.0,0.0,2,0,0,0,0.0,11.5,11.5,0,0,0,0.0,0,0,8.0,14,0,0.0,2
4,bbx_005,11.5,2.0,0.0,0,2.0,2.0,11.5,0.0,2.0,2.0,0.0,2.0,11.5,2,2.0,11.5,11.5,2,0,0,11.5,11.5,2,0,2,0,0,2,0,2,0,2,2,2,0,0,0,11.5,2.0,0,11.5,2.0,0,2,2.0,2.0,0.0,0.0,0,0.0,2.0,0,2.0,0,8.0,2,11.5,0.0,0,0,0,0,11.5,0.0,8.0,0,2,0,11.5,0,0,8.0,8,2,0.0,0


Generated Data

In [234]:
display(ffq_reverse_data.head())

,w1Bx_ffq_1,w1Bx_ffq_2,w1Bx_ffq_3,w1Bx_ffq_4,w1Bx_ffq_5,w1Bx_ffq_6,w1Bx_ffq_7,w1Bx_ffq_8,w1Bx_ffq_9,w1Bx_ffq_10,w1Bx_ffq_11,w1Bx_ffq_12,w1Bx_ffq_13,w1Bx_ffq_14,w1Bx_ffq_15,w1Bx_ffq_16,w1Bx_ffq_17,w1Bx_ffq_18,w1Bx_ffq_19,w1Bx_ffq_20,w1Bx_ffq_21,w1Bx_ffq_22,w1Bx_ffq_23,w1Bx_ffq_24,w1Bx_ffq_25,w1Bx_ffq_26,w1Bx_ffq_27,w1Bx_ffq_28,w1Bx_ffq_29,w1Bx_ffq_30,w1Bx_ffq_31,w1Bx_ffq_32,w1Bx_ffq_33,w1Bx_ffq_34,w1Bx_ffq_35,w1Bx_ffq_36,w1Bx_ffq_37,w1Bx_ffq_38,w1Bx_ffq_39,w1Bx_ffq_40,w1Bx_ffq_41,w1Bx_ffq_42,w1Bx_ffq_43,w1Bx_ffq_44,w1Bx_ffq_45,w1Bx_ffq_46,w1Bx_ffq_47,w1Bx_ffq_48,w1Bx_ffq_49,w1Bx_ffq_50,w1Bx_ffq_51,w1Bx_ffq_52,w1Bx_ffq_53,w1Bx_ffq_54,w1Bx_ffq_55,w1Bx_ffq_56,w1Bx_ffq_57,w1Bx_ffq_58,w1Bx_ffq_59,w1Bx_ffq_60,w1Bx_ffq_61,w1Bx_ffq_62,w1Bx_ffq_63,w1Bx_ffq_64,w1Bx_ffq_65,w1Bx_ffq_66,w1Bx_ffq_67,w1Bx_ffq_68,w1Bx_ffq_69,w1Bx_ffq_70,w1Bx_ffq_71,w1Bx_ffq_72,w1Bx_ffq_73,w1Bx_ffq_74,w1Bx_ffq_75,w1Bx_ffq_76,null_ct
patID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
bbx_001,11.5,11.5,0.0,0.0,11.5,11.5,0.0,11.5,8.0,11.5,2.0,11.5,11.5,0.0,0.0,11.5,2.0,0.0,2.0,0.0,0.0,2.0,2.0,0.0,2.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,2.0,2.0,0.0,2.0,2.0,0.0,2.0,0.0,2.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,2.0,8.0,2.0,2.0,0.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,11.5,0.0,0.0,8.0,0.0,0
bbx_002,11.5,14.0,2.0,0.0,0.0,8.0,8.0,2.0,11.5,11.5,2.0,11.5,11.5,0.0,2.0,2.0,2.0,0.0,0.0,0.0,2.0,11.5,2.0,0.0,0.0,2.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,0.0,2.0,0.0,0.0,0.0,2.0,0.0,11.5,0.0,0.0,0.0,11.5,2.0,0.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,8.0,0.0,0.0,0.0,0
bbx_003,14.0,14.0,11.5,0.0,11.5,8.0,0.0,11.5,2.0,11.5,0.0,11.5,2.0,0.0,11.5,8.0,11.5,0.0,0.0,0.0,2.0,8.0,0.0,0.0,8.0,2.0,0.0,8.0,0.0,2.0,2.0,0.0,0.0,2.0,0.0,2.0,0.0,11.5,11.5,0.0,11.5,11.5,8.0,0.0,14.0,11.5,11.5,11.5,2.0,11.5,11.5,2.0,11.5,2.0,2.0,0.0,14.0,11.5,0.0,0.0,0.0,0.0,11.5,0.0,11.5,0.0,2.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,11.5,0.0,0
bbx_004,2.0,2.0,2.0,0.0,11.5,8.0,8.0,2.0,2.0,11.5,11.5,2.0,2.0,2.0,0.0,2.0,2.0,0.0,0.0,0.0,0.0,11.5,2.0,0.0,0.0,2.0,2.0,2.0,2.0,2.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,11.5,0.0,0.0,0.0,0.0,2.0,11.5,2.0,0.0,2.0,0.0,0.0,2.0,0.0,11.5,0.0,11.5,2.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,11.5,11.5,0.0,0.0,0.0,0.0,0.0,0.0,8.0,14.0,0.0,0.0,2.0,0
bbx_005,11.5,2.0,0.0,0.0,2.0,2.0,11.5,0.0,2.0,2.0,0.0,2.0,11.5,2.0,2.0,11.5,11.5,2.0,0.0,0.0,11.5,11.5,2.0,0.0,2.0,0.0,0.0,2.0,0.0,2.0,0.0,2.0,2.0,2.0,0.0,0.0,0.0,11.5,2.0,0.0,11.5,2.0,0.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0,2.0,0.0,2.0,0.0,8.0,2.0,11.5,0.0,0.0,0.0,0.0,0.0,11.5,0.0,8.0,0.0,2.0,0.0,11.5,0.0,0.0,8.0,8.0,2.0,0.0,0.0,0


In [235]:
nutrient_info_updated

,ffq1,ffq2,ffq3,ffq4,ffq5,ffq6,ffq7,ffq8,ffq9,ffq10,ffq11,ffq12,ffq13,ffq14,ffq15,ffq16,ffq17,ffq18,ffq19,ffq20,ffq21,ffq22,ffq23,ffq24,ffq25,ffq26,ffq27,ffq28,ffq29,ffq30,ffq31,ffq32,ffq33,ffq34,ffq35,ffq36,ffq37,ffq38,ffq39,ffq40,ffq41,ffq42,ffq43,ffq44,ffq45,ffq46,ffq47,ffq48,ffq49,ffq50,ffq51,ffq52,ffq53,ffq54,ffq55,ffq56,ffq57,ffq58,ffq59,ffq60,ffq61,ffq62,ffq63,ffq64,ffq65,ffq66,ffq67,ffq68,ffq69,ffq70,ffq71,ffq72,ffq73,ffq74,ffq75,ffq76
kcal/portion,81.399978,40.256250,165.000000,108.2900,3.4500,15.00,129.789000,66.01500,13.253324,17.010000,11.200000,149.511193,253.952267,153.657000,141.466500,181.35000,240.684178,313.299257,278.691900,437.00,472.00000,170.00,342.9000,255.1100,348.032500,497.0,211.1200,113.00,191.6750,192.647500,376.2000,404.25000,626.0400,150.688125,345.7800,320.0400,178.9200,233.173750,142.759914,174.390996,279.999975,45.202500,363.200,181.073967,192.3200,71.700,259.066689,115.903227,119.148750,146.712500,90.8900,252.44400,205.9000,114.480,203.777939,269.690000,212.654263,259.910931,200.294794,26.7300,148.840,123.0000,102.4800,173.400,225.6800,122.400,83.700,115.416988,145.139877,0.0,132.457500,216.580000,2.3700,637.530,2.37,24.37500
fatg/portion,0.263070,0.169500,0.962437,1.3923,0.0585,0.15,12.591315,0.53865,0.118333,0.163800,0.130648,4.167576,11.366250,10.319400,6.446790,0.29295,10.373442,15.905231,14.810132,33.82,29.45072,5.22,16.3830,9.8746,15.140625,35.5,11.1839,2.73,9.4915,3.846375,16.3944,14.92425,26.4402,11.454300,18.9501,5.9976,4.9896,2.381838,1.851840,4.712421,10.487475,1.375500,13.052,14.456577,16.9424,8.091,6.333711,0.762999,0.914287,2.097625,6.6978,19.87707,10.1616,4.032,10.562214,11.086250,10.977025,12.730659,2.120769,0.0792,7.930,4.8708,2.3668,1.836,18.5584,1.734,0.372,0.010987,0.049077,0.0,0.121800,0.000000,0.0474,24.885,0.00,2.31075
sugarg/portion,13.307663,8.549156,6.268687,0.5474,0.0630,0.78,1.314225,3.63285,1.074466,1.963521,1.571752,1.420309,0.722021,0.104895,0.343035,0.00000,0.000000,5.530199,0.000000,0.00,0.21872,0.00,6.6548,5.7715,2.123725,0.0,0.0000,0.00,0.0000,2.413025,8.4546,5.34600,7.2372,0.450563,0.0000,7.9884,1.9908,1.434112,1.868440,0.179891,16.312500,0.103425,1.320,1.330961,2.4784,0.003,8.888689,5.280427,7.749000,0.363750,0.8723,0.00000,20.2130,17.280,8.400006,9.588475,23.732048,27.969743,32.667801,4.8081,12.322,12.4476,12.6880,32.385,0.2912,4.624,15.624,28.332624,35.116623,0.0,32.444475,0.479206,0.0000,71.100,0.00,0.02250


---

FFQ Generated Summary Calculations-  
Compare to the test data to validate.

In [236]:
# View FFQ Calculations
scoring.loc[scoring['Set']=="FFQ"]

,Set,Output,Calculation,Description,Notes
0,FFQ,w1ffq##_kcal,w1Bx_ffq_## * kcal_ffq##,Total kcal consumed for each food variable,kcal_ffq## comes from FFQ excel sheet; * assuming reverse scoring completed
1,FFQ,w1ffq_avgkcal,SUM(w1ffq##_kcal) / 14,Average kcal consumed each day for the reported 14 days,NaN
2,FFQ,w1ffq##_gfat,w1Bx_ffq_## * fat_ffq##,Total fat grams consumed for each food variable,fat_ffq## comes from FFQ excel sheet; * assuming reverse scoring completed
3,FFQ,w1ffq##_fatkcal,w1ffq##_gfat*9,Total kcal derived from fat consumed for each food variable,NaN
4,FFQ,w1ffq_avgfatkcal,SUM(w1ffq##_fatkcal)/14,Average kcal from fat consumed each day for the reported 14 days,NaN
5,FFQ,w1ffq_pctfat,SUM(w1ffq##_fatkcal)/SUM(w1ffq##_kcal)*100,Percent of total kcal derived from fat over 14 days,NaN
6,FFQ,w1ffq_pctfat_d,(w1ffq_avgfatkcal/w1ffq_avgkcal)*100,Daily Percent of total kcal derived from fat,NaN
7,FFQ,w1ffq##_gsug,w1Bx_ffq_## * sug_ffq##,Total sugar grams consumed for each food variable,sug_ffq## comes from FFQ excel sheet; * assuming reverse scoring completed
8,FFQ,w1ffq##_sugkcal,w1ffq##_gsug*4,Total kcal derived from sugar consumed for each food variable,NaN
9,FFQ,w1ffq_avgsugkcal,SUM(w1ffq##_sugkcal)/14,Average kcal from sugar consumed each day for the reported 14 days,NaN


In [237]:
display(nutrient_info.head())

,grams/med portion,kcal/100g,Pro/100g,fat(g)/100g,CHO/100g,sugars/100g,Satfat(g)/g
FFQ_quest,,,,,,,
ffq1,123.3333,66.0,0.5700,0.2133,17.2933,10.7900,0.0540
ffq2,84.7500,47.5,0.7950,0.2000,11.9300,10.0875,0.0513
ffq3,187.5000,88.0,4.9567,0.5133,17.3100,3.3433,0.1100
ffq4,119.0000,91.0,5.4100,1.1700,15.2600,0.4600,0.3910
ffq5,15.0000,23.0,2.8600,0.3900,3.6300,0.4200,0.0600


In [238]:
display(ffq_summary_vars[['w1ffq_avgkcal',
       'w1ffq_avgfatkcal', 'w1ffq_avgsugkcal', 'w1ffq_pctfat',
       'w1ffq_pctfat_d', 'w1ffq_pctsug', 'w1ffq_pctsug_d', 'null_ct']].head())

,w1ffq_avgkcal,w1ffq_avgfatkcal,w1ffq_avgsugkcal,w1ffq_pctfat,w1ffq_pctfat_d,w1ffq_pctsug,w1ffq_pctsug_d,null_ct
bbx_001,2184.258927,429.779259,173.146628,19.676205,19.676205,7.927019,7.927019,0
bbx_002,2347.075374,559.402543,202.394495,23.834025,23.834025,8.623264,8.623264,0
bbx_003,6136.911988,1424.173310,603.774121,23.206676,23.206676,9.838403,9.838403,0
bbx_004,3038.261620,793.942519,291.166625,26.131473,26.131473,9.583330,9.583330,0
bbx_005,4740.548849,1146.824586,461.018021,24.191810,24.191810,9.724993,9.724993,0


---

**kcal**

In [239]:
display(ffq_summary_vars[['w1ffq_avgkcal']].head())
display(ffq_kcal_test[['patID','w1ffq_avgkcal']])

,w1ffq_avgkcal
bbx_001,2184.258927
bbx_002,2347.075374
bbx_003,6136.911988
bbx_004,3038.261620
bbx_005,4740.548849


,patID,w1ffq_avgkcal
0,bbx_001,1581.333041
1,bbx_002,1585.278336
2,bbx_003,4108.964556
3,bbx_004,1953.152476
4,bbx_005,3132.706242


In [240]:
display(ffq_kcal_test)

,patID,w1Bx_ffq_1,w1Bx_ffq_2,w1Bx_ffq_3,w1Bx_ffq_4,w1Bx_ffq_5,w1Bx_ffq_6,w1Bx_ffq_7,w1Bx_ffq_8,w1Bx_ffq_9,w1Bx_ffq_10,w1Bx_ffq_11,w1Bx_ffq_12,w1Bx_ffq_13,w1Bx_ffq_14,w1Bx_ffq_15,w1Bx_ffq_16,w1Bx_ffq_17,w1Bx_ffq_18,w1Bx_ffq_19,w1Bx_ffq_20,w1Bx_ffq_21,w1Bx_ffq_22,w1Bx_ffq_23,w1Bx_ffq_24,w1Bx_ffq_25,w1Bx_ffq_26,w1Bx_ffq_27,w1Bx_ffq_28,w1Bx_ffq_29,w1Bx_ffq_30,w1Bx_ffq_31,w1Bx_ffq_32,w1Bx_ffq_33,w1Bx_ffq_34,w1Bx_ffq_35,w1Bx_ffq_36,w1Bx_ffq_37,w1Bx_ffq_38,w1Bx_ffq_39,w1Bx_ffq_40,w1Bx_ffq_41,w1Bx_ffq_42,w1Bx_ffq_43,w1Bx_ffq_44,w1Bx_ffq_45,w1Bx_ffq_46,w1Bx_ffq_47,w1Bx_ffq_48,w1Bx_ffq_49,w1Bx_ffq_50,w1Bx_ffq_51,w1Bx_ffq_52,w1Bx_ffq_53,w1Bx_ffq_54,w1Bx_ffq_55,w1Bx_ffq_56,w1Bx_ffq_57,w1Bx_ffq_58,w1Bx_ffq_59,w1Bx_ffq_60,w1Bx_ffq_61,w1Bx_ffq_62,w1Bx_ffq_63,w1Bx_ffq_64,w1Bx_ffq_65,w1Bx_ffq_66,w1Bx_ffq_67,w1Bx_ffq_68,w1Bx_ffq_69,w1Bx_ffq_70,w1Bx_ffq_71,w1Bx_ffq_72,w1Bx_ffq_73,w1Bx_ffq_74,w1Bx_ffq_75,w1Bx_ffq_76,w1ffq_avgkcal
0,bbx_001,936.099747,462.946875,0.0,0,39.675,172.5,0.0000,759.1725,106.026592,195.615,22.4,1719.378724,2920.451072,0.000,0.00000,2085.525,481.368355,0.000000,557.3838,0,0,340,685.8,0,696.065,0,0.00,226,0.00,0.000,0.0,0.0,1252.08,301.37625,0,640.08,357.84,0.000000,285.519828,0,559.999950,0.00000,0.0,0.000000,0.00,143.40,0.000000,0.000000,0.0000,293.42500,727.120,504.888,411.80,0.00,407.555878,539.38,0.000000,0.000000,0.000000,0,0,0,0.00,346.8,451.36,0,0.0,0,0.000000,0,0,2490.67,0.00,0.00,18.960,0.00,1581.333041
1,bbx_002,936.099747,563.587500,330.0,0,0.000,120.0,1038.3120,132.0300,152.413226,195.615,22.4,1719.378724,2920.451072,0.000,282.93300,362.700,481.368355,0.000000,0.0000,0,944,1955,685.8,0,0.000,994,0.00,0,0.00,385.295,0.0,0.0,0.00,0.00000,0,0.00,0.00,466.347500,285.519828,0,559.999950,0.00000,0.0,0.000000,384.64,0.00,2979.266923,0.000000,0.0000,0.00000,1045.235,504.888,0.00,228.96,407.555878,539.38,0.000000,0.000000,0.000000,0,0,0,204.96,346.8,0.00,0,0.0,0,0.000000,0,0,0.00,18.96,0.00,0.000,0.00,1585.278336
2,bbx_003,1139.599692,563.587500,1897.5,0,39.675,120.0,0.0000,759.1725,26.506648,195.615,0.0,1719.378724,507.904534,0.000,1626.86475,1450.800,2767.868044,0.000000,0.0000,0,944,1360,0.0,0,2784.260,994,0.00,904,0.00,385.295,752.4,0.0,0.00,301.37625,0,640.08,0.00,2681.498125,1641.739011,0,3219.999712,519.82875,2905.6,0.000000,2692.48,824.55,2979.266923,1332.887111,238.2975,1687.19375,1045.235,504.888,2367.85,228.96,407.555878,0.00,2977.159680,2988.975705,0.000000,0,0,0,1178.52,0.0,2595.32,0,167.4,0,0.000000,0,0,433.16,0.00,0.00,27.255,0.00,4108.964556
3,bbx_004,162.799956,80.512500,330.0,0,39.675,120.0,1038.3120,132.0300,26.506648,195.615,128.8,299.022387,507.904534,307.314,0.00000,362.700,481.368355,0.000000,0.0000,0,0,1955,685.8,0,0.000,994,422.24,226,383.35,385.295,0.0,0.0,1252.08,0.00000,0,0.00,0.00,0.000000,1641.739011,0,0.000000,0.00000,0.0,362.147935,2211.68,143.40,0.000000,231.806454,0.0000,0.00000,181.780,0.000,2367.85,0.00,2343.446297,539.38,0.000000,0.000000,400.589587,0,0,0,0.00,1994.1,2595.32,0,0.0,0,0.000000,0,0,1732.64,33.18,0.00,0.000,48.75,1953.152476
4,bbx_005,936.099747,80.512500,0.0,0,6.900,30.0,1492.5735,0.0000,26.506648,34.020,0.0,299.022387,2920.451072,307.314,282.93300,2085.525,2767.868044,626.598514,0.0000,0,5428,1955,685.8,0,696.065,0,0.00,226,0.00,385.295,0.0,808.5,1252.08,301.37625,0,0.00,0.00,2681.498125,285.519828,0,3219.999712,90.40500,0.0,362.147935,384.64,143.40,0.000000,0.000000,0.0000,0.00000,181.780,0.000,411.80,0.00,1630.223511,539.38,2445.524023,0.000000,0.000000,0,0,0,1178.52,0.0,1805.44,0,167.4,0,1669.108586,0,0,1732.64,18.96,1275.06,0.000,0.00,3132.706242


In [241]:
display(ffq_summary_vars.filter(like="_kcal").head())

,w1Bx_ffq_1_kcal,w1Bx_ffq_2_kcal,w1Bx_ffq_3_kcal,w1Bx_ffq_4_kcal,w1Bx_ffq_5_kcal,w1Bx_ffq_6_kcal,w1Bx_ffq_7_kcal,w1Bx_ffq_8_kcal,w1Bx_ffq_9_kcal,w1Bx_ffq_10_kcal,w1Bx_ffq_11_kcal,w1Bx_ffq_12_kcal,w1Bx_ffq_13_kcal,w1Bx_ffq_14_kcal,w1Bx_ffq_15_kcal,w1Bx_ffq_16_kcal,w1Bx_ffq_17_kcal,w1Bx_ffq_18_kcal,w1Bx_ffq_19_kcal,w1Bx_ffq_20_kcal,w1Bx_ffq_21_kcal,w1Bx_ffq_22_kcal,w1Bx_ffq_23_kcal,w1Bx_ffq_24_kcal,w1Bx_ffq_25_kcal,w1Bx_ffq_26_kcal,w1Bx_ffq_27_kcal,w1Bx_ffq_28_kcal,w1Bx_ffq_29_kcal,w1Bx_ffq_30_kcal,w1Bx_ffq_31_kcal,w1Bx_ffq_32_kcal,w1Bx_ffq_33_kcal,w1Bx_ffq_34_kcal,w1Bx_ffq_35_kcal,w1Bx_ffq_36_kcal,w1Bx_ffq_37_kcal,w1Bx_ffq_38_kcal,w1Bx_ffq_39_kcal,w1Bx_ffq_40_kcal,w1Bx_ffq_41_kcal,w1Bx_ffq_42_kcal,w1Bx_ffq_43_kcal,w1Bx_ffq_44_kcal,w1Bx_ffq_45_kcal,w1Bx_ffq_46_kcal,w1Bx_ffq_47_kcal,w1Bx_ffq_48_kcal,w1Bx_ffq_49_kcal,w1Bx_ffq_50_kcal,w1Bx_ffq_51_kcal,w1Bx_ffq_52_kcal,w1Bx_ffq_53_kcal,w1Bx_ffq_54_kcal,w1Bx_ffq_55_kcal,w1Bx_ffq_56_kcal,w1Bx_ffq_57_kcal,w1Bx_ffq_58_kcal,w1Bx_ffq_59_kcal,w1Bx_ffq_60_kcal,w1Bx_ffq_61_kcal,w1Bx_ffq_62_kcal,w1Bx_ffq_63_kcal,w1Bx_ffq_64_kcal,w1Bx_ffq_65_kcal,w1Bx_ffq_66_kcal,w1Bx_ffq_67_kcal,w1Bx_ffq_68_kcal,w1Bx_ffq_69_kcal,w1Bx_ffq_70_kcal,w1Bx_ffq_71_kcal,w1Bx_ffq_72_kcal,w1Bx_ffq_73_kcal,w1Bx_ffq_74_kcal,w1Bx_ffq_75_kcal,w1Bx_ffq_76_kcal
bbx_001,936.099747,462.946875,0.0,0.0,39.675,172.5,0.0000,759.1725,106.026592,195.615,22.4,1719.378724,2920.451072,0.000,0.00000,2085.525,481.368355,0.000000,557.3838,0.0,0.0,340.0,685.8,0.0,696.065,0.0,0.00,226.0,0.00,0.000,0.0,0.0,1252.08,301.37625,0.0,640.08,357.84,0.000000,285.519828,0.0,559.999950,0.00000,0.0,0.000000,0.00,143.40,0.000000,0.000000,0.0000,293.42500,727.120,504.888,411.80,0.00,407.555878,539.38,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.00,346.8,451.36,0.0,0.0,0.0,0.000000,0.0,0.0,2490.67,0.00,0.00,18.960,0.00
bbx_002,936.099747,563.587500,330.0,0.0,0.000,120.0,1038.3120,132.0300,152.413226,195.615,22.4,1719.378724,2920.451072,0.000,282.93300,362.700,481.368355,0.000000,0.0000,0.0,944.0,1955.0,685.8,0.0,0.000,994.0,0.00,0.0,0.00,385.295,0.0,0.0,0.00,0.00000,0.0,0.00,0.00,466.347500,285.519828,0.0,559.999950,0.00000,0.0,0.000000,384.64,0.00,2979.266923,0.000000,0.0000,0.00000,1045.235,504.888,0.00,228.96,407.555878,539.38,0.000000,0.000000,0.000000,0.0,0.0,0.0,204.96,346.8,0.00,0.0,0.0,0.0,0.000000,0.0,0.0,0.00,18.96,0.00,0.000,0.00
bbx_003,1139.599692,563.587500,1897.5,0.0,39.675,120.0,0.0000,759.1725,26.506648,195.615,0.0,1719.378724,507.904534,0.000,1626.86475,1450.800,2767.868044,0.000000,0.0000,0.0,944.0,1360.0,0.0,0.0,2784.260,994.0,0.00,904.0,0.00,385.295,752.4,0.0,0.00,301.37625,0.0,640.08,0.00,2681.498125,1641.739011,0.0,3219.999712,519.82875,2905.6,0.000000,2692.48,824.55,2979.266923,1332.887111,238.2975,1687.19375,1045.235,504.888,2367.85,228.96,407.555878,0.00,2977.159680,2988.975705,0.000000,0.0,0.0,0.0,1178.52,0.0,2595.32,0.0,167.4,0.0,0.000000,0.0,0.0,433.16,0.00,0.00,27.255,0.00
bbx_004,162.799956,80.512500,330.0,0.0,39.675,120.0,1038.3120,132.0300,26.506648,195.615,128.8,299.022387,507.904534,307.314,0.00000,362.700,481.368355,0.000000,0.0000,0.0,0.0,1955.0,685.8,0.0,0.000,994.0,422.24,226.0,383.35,385.295,0.0,0.0,1252.08,0.00000,0.0,0.00,0.00,0.000000,1641.739011,0.0,0.000000,0.00000,0.0,362.147935,2211.68,143.40,0.000000,231.806454,0.0000,0.00000,181.780,0.000,2367.85,0.00,2343.446297,539.38,0.000000,0.000000,400.589587,0.0,0.0,0.0,0.00,1994.1,2595.32,0.0,0.0,0.0,0.000000,0.0,0.0,1732.64,33.18,0.00,0.000,48.75
bbx_005,936.099747,80.512500,0.0,0.0,6.900,30.0,1492.5735,0.0000,26.506648,34.020,0.0,299.022387,2920.451072,307.314,282.93300,2085.525,2767.868044,626.598514,0.0000,0.0,5428.0,1955.0,685.8,0.0,696.065,0.0,0.00,226.0,0.00,385.295,0.0,808.5,1252.08,301.37625,0.0,0.00,0.00,2681.498125,285.519828,0.0,3219.999712,90.40500,0.0,362.147935,384.64,143.40,0.000000,0.000000,0.0000,0.00000,181.780,0.000,411.80,0.00,1630.223511,539.38,2445.524023,0.000000,0.000000,0.0,0.0,0.0,1178.52,0.0,1805.44,0.0,167.4,0.0,1669.108585,0.0,0.0,1732.64,1

---

**avgfatkcal**

In [242]:
display(ffq_summary_vars[['w1ffq_avgfatkcal']].head())
display(ffq_fatkcal_test[['patID','w1ffq_avgfatkcal']])

,w1ffq_avgfatkcal
bbx_001,429.779259
bbx_002,559.402543
bbx_003,1424.173310
bbx_004,793.942519
bbx_005,1146.824586


,patID,w1ffq_avgfatkcal
0,bbx_001,429.779259
1,bbx_002,559.402543
2,bbx_003,1424.173310
3,bbx_004,793.942519
4,bbx_005,1146.824586


---

**pctfat**

In [243]:
display(ffq_summary_vars[['w1ffq_pctfat']].head())
display(ffq_summary_vars[['w1ffq_pctfat_d']].head())
display(ffq_fatkcal_test[['patID','w1ffq_pctfat']])

,w1ffq_pctfat
bbx_001,19.676205
bbx_002,23.834025
bbx_003,23.206676
bbx_004,26.131473
bbx_005,24.191810


,w1ffq_pctfat_d
bbx_001,19.676205
bbx_002,23.834025
bbx_003,23.206676
bbx_004,26.131473
bbx_005,24.191810


,patID,w1ffq_pctfat
0,bbx_001,27.178289
1,bbx_002,35.287339
2,bbx_003,34.660151
3,bbx_004,40.649285
4,bbx_005,36.608111


---  
**avgsugkcal**

In [244]:
display(ffq_summary_vars[['w1ffq_avgsugkcal']].head())
display(ffq_sugkcal_test[['patID','w1ffq_avgsugkcal']])

,w1ffq_avgsugkcal
bbx_001,173.146628
bbx_002,202.394495
bbx_003,603.774121
bbx_004,291.166625
bbx_005,461.018021


,patID,w1ffq_avgsugkcal
0,bbx_001,173.146628
1,bbx_002,202.394495
2,bbx_003,603.774121
3,bbx_004,291.166625
4,bbx_005,461.018021


---  
**pctsug**

In [245]:
display(ffq_summary_vars[['w1ffq_pctsug']].head())
display(ffq_summary_vars[['w1ffq_pctsug_d']].head())
display(ffq_sugkcal_test[['patID','w1ffq_pctsug']])

,w1ffq_pctsug
bbx_001,7.927019
bbx_002,8.623264
bbx_003,9.838403
bbx_004,9.583330
bbx_005,9.724993


,w1ffq_pctsug_d
bbx_001,7.927019
bbx_002,8.623264
bbx_003,9.838403
bbx_004,9.583330
bbx_005,9.724993


,patID,w1ffq_pctsug
0,bbx_001,10.949409
1,bbx_002,12.767127
2,bbx_003,14.694070
3,bbx_004,14.907521
4,bbx_005,14.716286


In [197]:
display(ffq_sugkcal_test)

,patID,w1Bx_ffq_1,w1Bx_ffq_2,w1Bx_ffq_3,w1Bx_ffq_4,w1Bx_ffq_5,w1Bx_ffq_6,w1Bx_ffq_7,w1Bx_ffq_8,w1Bx_ffq_9,w1Bx_ffq_10,w1Bx_ffq_11,w1Bx_ffq_12,w1Bx_ffq_13,w1Bx_ffq_14,w1Bx_ffq_15,w1Bx_ffq_16,w1Bx_ffq_17,w1Bx_ffq_18,w1Bx_ffq_19,w1Bx_ffq_20,w1Bx_ffq_21,w1Bx_ffq_22,w1Bx_ffq_23,w1Bx_ffq_24,w1Bx_ffq_25,w1Bx_ffq_26,w1Bx_ffq_27,w1Bx_ffq_28,w1Bx_ffq_29,w1Bx_ffq_30,w1Bx_ffq_31,w1Bx_ffq_32,w1Bx_ffq_33,w1Bx_ffq_34,w1Bx_ffq_35,w1Bx_ffq_36,w1Bx_ffq_37,w1Bx_ffq_38,w1Bx_ffq_39,w1Bx_ffq_40,w1Bx_ffq_41,w1Bx_ffq_42,w1Bx_ffq_43,w1Bx_ffq_44,w1Bx_ffq_45,w1Bx_ffq_46,w1Bx_ffq_47,w1Bx_ffq_48,w1Bx_ffq_49,w1Bx_ffq_50,w1Bx_ffq_51,w1Bx_ffq_52,w1Bx_ffq_53,w1Bx_ffq_54,w1Bx_ffq_55,w1Bx_ffq_56,w1Bx_ffq_57,w1Bx_ffq_58,w1Bx_ffq_59,w1Bx_ffq_60,w1Bx_ffq_61,w1Bx_ffq_62,w1Bx_ffq_63,w1Bx_ffq_64,w1Bx_ffq_65,w1Bx_ffq_66,w1Bx_ffq_67,w1Bx_ffq_68,w1Bx_ffq_69,w1Bx_ffq_70,w1Bx_ffq_71,w1Bx_ffq_72,w1Bx_ffq_73,w1Bx_ffq_74,w1Bx_ffq_75,w1Bx_ffq_76,w1ffq_avgsugg,w1ffq_avgsugkcal,w1ffq_pctsug,w1ffq_pctsug_d
0,bbx_001,153.038125,98.315297,0.000000,0,0.7245,8.97,0.000000,41.777775,8.595727,22.580491,3.143504,16.333555,8.303236,0.00000,0.000000,0,0,0.000000,0,0,0.00000,0,13.3096,0,4.24745,0,0,0,0,0.00000,0.0000,0.000,14.4744,0.901125,0,15.9768,3.9816,0.000000,3.736879,0,32.62500,0.000000,0.00,0.000000,0.0000,0.0060,0.000000,0.000000,0.000,0.727500,6.97840,0,40.4260,0.00,16.800012,19.17695,0.000000,0.000000,0.000000,0,0,0,0.000,64.7700,0.5824,0,0.000,0,0.000000,0,0,5.510869,0,0.0,0,0.000,43.286657,173.146628,10.949409,10.949409
1,bbx_002,153.038125,119.688187,12.537375,0,0.0000,6.24,10.513800,7.265700,12.356358,22.580491,3.143504,16.333555,8.303236,0.00000,0.686070,0,0,0.000000,0,0,0.43744,0,13.3096,0,0.00000,0,0,0,0,4.82605,0.0000,0.000,0.0000,0.000000,0,0.0000,0.0000,2.868225,3.736879,0,32.62500,0.000000,0.00,0.000000,4.9568,0.0000,102.219924,0.000000,0.000,0.000000,10.03145,0,0.0000,34.56,16.800012,19.17695,0.000000,0.000000,0.000000,0,0,0,25.376,64.7700,0.0000,0,0.000,0,0.000000,0,0,0.000000,0,0.0,0,0.000,50.598624,202.394495,12.767127,12.767127
2,bbx_003,186.307283,119.688187,72.089906,0,0.7245,6.24,0.000000,41.777775,2.148932,22.580491,0.000000,16.333555,1.444041,0.00000,3.944902,0,0,0.000000,0,0,0.43744,0,0.0000,0,16.98980,0,0,0,0,4.82605,16.9092,0.000,0.0000,0.901125,0,15.9768,0.0000,16.492294,21.487055,0,187.59375,1.189387,10.56,0.000000,34.6976,0.0345,102.219924,60.724916,15.498,4.183125,10.03145,0,232.4495,34.56,16.800012,0.00000,332.248672,321.652039,0.000000,0,0,0,145.912,0.0000,3.3488,0,31.248,0,0.000000,0,0,0.958412,0,0.0,0,0.000,150.943530,603.774121,14.694070,14.694070
3,bbx_004,26.615326,17.098312,12.537375,0,0.7245,6.24,10.513800,7.265700,2.148932,22.580491,18.075148,2.840618,1.444041,0.20979,0.000000,0,0,0.000000,0,0,0.00000,0,13.3096,0,0.00000,0,0,0,0,4.82605,0.0000,0.000,14.4744,0.000000,0,0.0000,0.0000,0.000000,21.487055,0,0.00000,0.000000,0.00,2.661923,28.5016,0.0060,0.000000,10.560855,0.000,0.000000,1.74460,0,232.4495,0.00,96.600069,19.17695,0.000000,0.000000,65.335603,0,0,0,0.000,372.4275,3.3488,0,0.000,0,0.000000,0,0,3.833648,0,0.0,0,0.045,72.791656,291.166625,14.907521,14.907521
4,bbx_005,153.038125,17.098312,0.000000,0,0.1260,1.56,15.113587,0.000000,2.148932,3.927042,0.000000,2.840618,8.303236,0.20979,0.686070,0,0,11.060398,0,0,2.51528,0,13.3096,0,4.24745,0,0,0,0,4.82605,0.0000,10.692,14.4744,0.901125,0,0.0000,0.0000,16.492294,3.736879,0,187.59375,0.206850,0.00,2.661923,4.9568,0.0060,0.000000,0.000000,0.000,0.000000,1.74460,0,40.4260,0.00,67.200048,19.17695,272.918552,0.000000,0.000000,0,0,0,145.912,0.0000,2.3296,0,31.248,0,403.841164,0,0,3.833648,0,142.2,0,0.000,115.254505,461.018021,14.716286,14.716286


In [198]:
display(ffq_summary_vars.filter(like="sugkcal", axis=1).head())

,w1Bx_ffq_1_sugkcal,w1Bx_ffq_2_sugkcal,w1Bx_ffq_3_sugkcal,w1Bx_ffq_4_sugkcal,w1Bx_ffq_5_sugkcal,w1Bx_ffq_6_sugkcal,w1Bx_ffq_7_sugkcal,w1Bx_ffq_8_sugkcal,w1Bx_ffq_9_sugkcal,w1Bx_ffq_10_sugkcal,w1Bx_ffq_11_sugkcal,w1Bx_ffq_12_sugkcal,w1Bx_ffq_13_sugkcal,w1Bx_ffq_14_sugkcal,w1Bx_ffq_15_sugkcal,w1Bx_ffq_16_sugkcal,w1Bx_ffq_17_sugkcal,w1Bx_ffq_18_sugkcal,w1Bx_ffq_19_sugkcal,w1Bx_ffq_20_sugkcal,w1Bx_ffq_21_sugkcal,w1Bx_ffq_22_sugkcal,w1Bx_ffq_23_sugkcal,w1Bx_ffq_24_sugkcal,w1Bx_ffq_25_sugkcal,w1Bx_ffq_26_sugkcal,w1Bx_ffq_27_sugkcal,w1Bx_ffq_28_sugkcal,w1Bx_ffq_29_sugkcal,w1Bx_ffq_30_sugkcal,w1Bx_ffq_31_sugkcal,w1Bx_ffq_32_sugkcal,w1Bx_ffq_33_sugkcal,w1Bx_ffq_34_sugkcal,w1Bx_ffq_35_sugkcal,w1Bx_ffq_36_sugkcal,w1Bx_ffq_37_sugkcal,w1Bx_ffq_38_sugkcal,w1Bx_ffq_39_sugkcal,w1Bx_ffq_40_sugkcal,w1Bx_ffq_41_sugkcal,w1Bx_ffq_42_sugkcal,w1Bx_ffq_43_sugkcal,w1Bx_ffq_44_sugkcal,w1Bx_ffq_45_sugkcal,w1Bx_ffq_46_sugkcal,w1Bx_ffq_47_sugkcal,w1Bx_ffq_48_sugkcal,w1Bx_ffq_49_sugkcal,w1Bx_ffq_50_sugkcal,w1Bx_ffq_51_sugkcal,w1Bx_ffq_52_sugkcal,w1Bx_ffq_53_sugkcal,w1Bx_ffq_54_sugkcal,w1Bx_ffq_55_sugkcal,w1Bx_ffq_56_sugkcal,w1Bx_ffq_57_sugkcal,w1Bx_ffq_58_sugkcal,w1Bx_ffq_59_sugkcal,w1Bx_ffq_60_sugkcal,w1Bx_ffq_61_sugkcal,w1Bx_ffq_62_sugkcal,w1Bx_ffq_63_sugkcal,w1Bx_ffq_64_sugkcal,w1Bx_ffq_65_sugkcal,w1Bx_ffq_66_sugkcal,w1Bx_ffq_67_sugkcal,w1Bx_ffq_68_sugkcal,w1Bx_ffq_69_sugkcal,w1Bx_ffq_70_sugkcal,w1Bx_ffq_71_sugkcal,w1Bx_ffq_72_sugkcal,w1Bx_ffq_73_sugkcal,w1Bx_ffq_74_sugkcal,w1Bx_ffq_75_sugkcal,w1Bx_ffq_76_sugkcal,w1ffq_avgsugkcal
bbx_001,6.121525,3.932612,0.000000,0.0,0.02898,0.3588,0.000000,1.671111,0.343829,0.903220,0.125740,0.653342,0.332129,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.532384,0.0,0.169898,0.0,0.0,0.0,0.0,0.000000,0.000000,0.00000,0.578976,0.036045,0.0,0.639072,0.159264,0.000000,0.149475,0.0,1.30500,0.000000,0.0000,0.000000,0.000000,0.00024,0.000000,0.000000,0.00000,0.029100,0.279136,0.0,1.61704,0.0000,0.672000,0.767078,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.00000,2.5908,0.023296,0.0,0.00000,0.0,0.000000,0.0,0.0,0.220435,0.0,0.000,0.0,0.0000,1.731466
bbx_002,6.121525,4.787528,0.501495,0.0,0.00000,0.2496,0.420552,0.290628,0.494254,0.903220,0.125740,0.653342,0.332129,0.000000,0.027443,0.0,0.0,0.000000,0.0,0.0,0.017498,0.0,0.532384,0.0,0.000000,0.0,0.0,0.0,0.0,0.193042,0.000000,0.00000,0.000000,0.000000,0.0,0.000000,0.000000,0.114729,0.149475,0.0,1.30500,0.000000,0.0000,0.000000,0.198272,0.00000,4.088797,0.000000,0.00000,0.000000,0.401258,0.0,0.00000,1.3824,0.672000,0.767078,0.000000,0.000000,0.000000,0.0,0.0,0.0,1.01504,2.5908,0.000000,0.0,0.00000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000,0.0,0.0000,2.023945
bbx_003,7.452291,4.787528,2.883596,0.0,0.02898,0.2496,0.000000,1.671111,0.085957,0.903220,0.000000,0.653342,0.057762,0.000000,0.157796,0.0,0.0,0.000000,0.0,0.0,0.017498,0.0,0.000000,0.0,0.679592,0.0,0.0,0.0,0.0,0.193042,0.676368,0.00000,0.000000,0.036045,0.0,0.639072,0.000000,0.659692,0.859482,0.0,7.50375,0.047575,0.4224,0.000000,1.387904,0.00138,4.088797,2.428997,0.61992,0.167325,0.401258,0.0,9.29798,1.3824,0.672000,0.000000,13.289947,12.866082,0.000000,0.0,0.0,0.0,5.83648,0.0000,0.133952,0.0,1.24992,0.0,0.000000,0.0,0.0,0.038336,0.0,0.000,0.0,0.0000,6.037741
bbx_004,1.064613,0.683933,0.501495,0.0,0.02898,0.2496,0.420552,0.290628,0.085957,0.903220,0.723006,0.113625,0.057762,0.008392,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.532384,0.0,0.000000,0.0,0.0,0.0,0.0,0.193042,0.000000,0.00000,0.578976,0.000000,0.0,0.000000,0.000000,0.000000,0.859482,0.0,0.00000,0.000000,0.0000,0.106477,1.140064,0.00024,0.000000,0.422434,0.00000,0.000000,0.069784,0.0,9.29798,0.0000,3.864003,0.767078,0.000000,0.000000,2.613424,0.0,0.0,0.0,0.00000,14.8971,0.133952,0.0,0.00000,0.0,0.000000,0.0,0.0,0.153346,0.0,0.000,0.0,0.0018,2.911666
bbx_005,6.121525,0.683933,0.000000,0.0,0.00504,0.0624,0.604544,0.000000,0.085957,0.157082,0.000000,0.113625,0.332129,0.008392,0.027443,0.0,0.0,0.442416,0.0,0.0,0.100611,0.0,0.532384,0.0,0.169898,0.0,0.0,0.0,0

---

 
## BIS <a anchor="class" id="bis"></a>
--- 

In [16]:
# FFQ Calculations
scoring.loc[scoring['Set']=="BIS-15"]

,Set,Output,Calculation,Description,Notes
12,BIS-15,w1bis_tot,SUM(w1Bx_bis_##),Total score on BIS-15,* assuming reverse scoring completed
13,BIS-15,w1bis_npi,"SUM(w1Bx_bis_6, w1Bx_bis_7, w1Bx_bis_8, w1Bx_bis_9, w1Bx_bis_10)",Score on Non-Planning Impulsivity subscale of BIS-15,* assuming reverse scoring completed
14,BIS-15,w1bis_mi,"SUM(w1Bx_bis_1, w1Bx_bis_2, w1Bx_bis_3, w1Bx_bis_4, w1Bx_bis_5)",Score on Motor Impulsivity subscale of BIS-15,* assuming reverse scoring completed
15,BIS-15,w1bis_ai,"SUM(w1Bx_bis_11, w1Bx_bis_12, w1Bx_bis_13, w1Bx_bis_14, w1Bx_bis_15)",Score on Attentional Impulsivity subscale of BIS-15,* assuming reverse scoring completed


In [17]:
# Look at the filtered df
s1_behav.filter(regex="bis_[0-9]", axis=1).head()

,w1Bx_bis_1,w1Bx_bis_2,w1Bx_bis_3,w1Bx_bis_4,w1Bx_bis_5,w1Bx_bis_6,w1Bx_bis_7,w1Bx_bis_8,w1Bx_bis_9,w1Bx_bis_10,w1Bx_bis_11,w1Bx_bis_12,w1Bx_bis_13,w1Bx_bis_14,w1Bx_bis_15
patID,,,,,,,,,,,,,,,
bbx_001,2,2,1,1,2,4,4,3,3,3,2,1,2,2,1
bbx_002,3,3,2,3,2,3,4,4,3,3,2,2,2,1,1
bbx_003,2,1,1,1,1,3,4,4,4,3,3,2,2,1,2
bbx_004,1,2,1,1,1,4,4,4,4,4,2,2,3,2,2
bbx_005,2,2,2,1,1,4,4,4,3,3,2,1,2,3,3


In [18]:
bis_df=bis_calculation()


[INFO] ERROR with BIS-15 calculation for ids:  {'bbx_057'}

[INFO] completed BIS-15 calculations.



## FCI <a anchor="class" id="fci"></a> 
---

In [19]:
# FFQ Calculations
scoring.loc[scoring['Set']=="FCI"]

,Set,Output,Calculation,Description,Notes
16,FCI,w1fci_crave,MEAN(w1Bx_fciCrv_##),Food Craving Inventory: Total Food Craving Score,NaN
17,FCI,w1fci_chf,"MEAN(w1Bx_fciCrv_3 , w1Bx_fciCrv_6 , w1Bx_fciCrv_4 , w1Bx_fciCrv_15 , w1Bx_fciCrv_26 , w1Bx_fciCrv_19 , w1Bx_fciCrv_10 , w1Bx_fciCrv_27)",Food Craving Inventory: Crave High Fats,NaN
18,FCI,w1fci_cs,"MEAN(w1Bx_fciCrv_25 , w1Bx_fciCrv_16 , w1Bx_fciCrv_24 , w1Bx_fciCrv_17 , w1Bx_fciCrv_23 , w1Bx_fciCrv_1 , w1Bx_fciCrv_8 , w1Bx_fciCrv_13)",Food Craving Inventory: Crave Sweets,NaN
19,FCI,w1fci_cc,"MEAN(w1Bx_fciCrv_21 , w1Bx_fciCrv_18 , w1Bx_fciCrv_12 , w1Bx_fciCrv_5 , w1Bx_fciCrv_9 , w1Bx_fciCrv_28 , w1Bx_fciCrv_14 , w1Bx_fciCrv_22)",Food Craving Inventory: Crave Carbohydrates/Starches,NaN
20,FCI,w1fci_cff,"MEAN(w1Bx_fciCrv_11 , w1Bx_fciCrv_7 , w1Bx_fciCrv_20 , w1Bx_fciCrv_2)",Food Craving Inventory: Crave Fast-Food Fats,NaN
21,FCI,w1fci_like,MEAN(w1Bx_fciLik_##),Food Craving Inventory: Total Food Liking Score,NaN
22,FCI,w1fci_lhf,"MEAN(w1Bx_fciLik_3 , w1Bx_fciLik_6 , w1Bx_fciLik_4 , w1Bx_fciLik_15 , w1Bx_fciLik_26 , w1Bx_fciLik_19 , w1Bx_fciLik_10 , w1Bx_fciLik_27)",Food Craving Inventory: Like High Fats,NaN
23,FCI,w1fci_ls,"MEAN(w1Bx_fciLik_25 , w1Bx_fciLik_16 , w1Bx_fciLik_24 , w1Bx_fciLik_17 , w1Bx_fciLik_23 , w1Bx_fciLik_1 , w1Bx_fciLik_8 , w1Bx_fciLik_13)",Food Craving Inventory: Like Sweets,NaN
24,FCI,w1fci_lc,"MEAN(w1Bx_fciLik_21 , w1Bx_fciLik_18 , w1Bx_fciLik_12 , w1Bx_fciLik_5 , w1Bx_fciLik_9 , w1Bx_fciLik_28 , w1Bx_fciLik_14 , w1Bx_fciLik_22)",Food Craving Inventory: Like Carbohydrates/Starches,NaN
25,FCI,w1fci_lff,"MEAN(w1Bx_fciLik_11 , w1Bx_fciLik_7 , w1Bx_fciLik_20 , w1Bx_fciLik_2)",Food Craving Inventory: Like Fast-Food Fats,NaN


In [20]:
# Look at the filtered df (the input data)
s1_behav.filter(like="fciCrv", axis=1).head()

,w1Bx_fciCrv_1,w1Bx_fciCrv_2,w1Bx_fciCrv_3,w1Bx_fciCrv_4,w1Bx_fciCrv_5,w1Bx_fciCrv_6,w1Bx_fciCrv_7,w1Bx_fciCrv_8,w1Bx_fciCrv_9,w1Bx_fciCrv_10,w1Bx_fciCrv_11,w1Bx_fciCrv_12,w1Bx_fciCrv_13,w1Bx_fciCrv_14,w1Bx_fciCrv_15,w1Bx_fciCrv_16,w1Bx_fciCrv_17,w1Bx_fciCrv_18,w1Bx_fciCrv_19,w1Bx_fciCrv_20,w1Bx_fciCrv_21,w1Bx_fciCrv_22,w1Bx_fciCrv_23,w1Bx_fciCrv_24,w1Bx_fciCrv_25,w1Bx_fciCrv_26,w1Bx_fciCrv_27,w1Bx_fciCrv_28
patID,,,,,,,,,,,,,,,,,,,,,,,,,,,,
bbx_001,2,3,3,1,2,3,3,1,2,3,3,2,4,3,1,2,3,2,2,2,2,2,3,2,2,3,2,1
bbx_002,3,3,1,1,1,1,1,2,1,2,3,1,3,3,1,3,1,1,1,1,1,1,1,1,1,1,1,1
bbx_003,2,1,1,1,2,3,1,4,1,1,1,3,5,2,1,4,5,4,1,1,1,4,3,1,3,1,3,1
bbx_004,3,3,1,1,1,1,2,2,1,1,3,1,4,2,1,4,3,2,1,1,1,1,1,3,3,2,3,1
bbx_005,3,3,1,1,1,1,3,3,2,1,3,3,4,3,1,4,4,3,1,2,3,1,4,4,4,2,2,1


In [21]:
s1_behav.filter(like="fciLik", axis=1).head()

,w1Bx_fciLik_1,w1Bx_fciLik_2,w1Bx_fciLik_3,w1Bx_fciLik_4,w1Bx_fciLik_5,w1Bx_fciLik_6,w1Bx_fciLik_7,w1Bx_fciLik_8,w1Bx_fciLik_9,w1Bx_fciLik_10,w1Bx_fciLik_11,w1Bx_fciLik_12,w1Bx_fciLik_13,w1Bx_fciLik_14,w1Bx_fciLik_15,w1Bx_fciLik_16,w1Bx_fciLik_17,w1Bx_fciLik_18,w1Bx_fciLik_19,w1Bx_fciLik_20,w1Bx_fciLik_21,w1Bx_fciLik_22,w1Bx_fciLik_23,w1Bx_fciLik_24,w1Bx_fciLik_25,w1Bx_fciLik_26,w1Bx_fciLik_27,w1Bx_fciLik_28
patID,,,,,,,,,,,,,,,,,,,,,,,,,,,,
bbx_001,2,4,3,2,2,3,3,2,3,3,3,3,3,3,2,3,3,2,2,2,2,2,2,2,2,4,2,1
bbx_002,3,3,3,3,1,2,3,4,2,2,2,3,2,3,1,3,4,4,3,1,3,1,1,1,3,3,3,2
bbx_003,2,1,2,1,2,3,1,4,2,2,1,4,4,2,1,4,4,4,2,1,2,4,3,3,4,2,4,2
bbx_004,2,2,1,1,2,1,2,2,2,1,2,2,4,2,2,3,3,2,2,2,2,3,2,3,3,2,3,2
bbx_005,4,4,3,1,1,2,2,3,3,1,3,3,4,3,2,4,4,3,1,2,3,1,4,3,4,2,3,2


In [22]:
fci_df=fci_calculation()

,w1fci_crave,w1fci_chf,w1fci_cs,w1fci_cc,w1fci_cff,w1fci_like,w1fci_lhf,w1fci_ls,w1fci_lc,w1fci_lff
bbx_166,3.035714,2.375,3.375,2.875,4.00,3.035714,2.500,3.250,3.000,3.75
bbx_167,1.678571,2.000,1.875,1.250,1.50,2.071429,2.250,2.000,1.875,2.25
bbx_168,2.607143,1.875,3.000,2.750,3.00,2.607143,2.000,3.000,2.750,2.75
bbx_169,1.535714,1.000,2.000,1.250,2.25,2.571429,2.000,3.125,2.500,2.75
bbx_170,2.071429,1.625,2.875,1.500,2.50,2.892857,2.625,3.375,2.500,3.25



[INFO] completed FCI calculations.



## DEBQ<a anchor="class" id="debq"></a>
---

In [23]:
# debq Calculations
scoring.loc[scoring['Set']=="DEBQ"]

,Set,Output,Calculation,Description,Notes
26,DEBQ,w1debq_tot,MEAN(w1Bx_debq_##),Total score on DEBQ,* assuming reverse scoring completed
27,DEBQ,w1debq_r,"MEAN(w1Bx_debq_1 , w1Bx_debq_2 , w1Bx_debq_3 , w1Bx_debq_4 , w1Bx_debq_5 , w1Bx_debq_6 , w1Bx_debq_7 , w1Bx_debq_8 , w1Bx_debq_9 , w1Bx_debq_10)",DEBQ: Restrained Eating,* assuming reverse scoring completed
28,DEBQ,w1debq_e,"MEAN(w1Bx_debq_11 , w1Bx_debq_12 , w1Bx_debq_13 , w1Bx_debq_14 , w1Bx_debq_15 , w1Bx_debq_16 , w1Bx_debq_17 , w1Bx_debq_18 , w1Bx_debq_19 , w1Bx_debq_20 , w1Bx_debq_21 , w1Bx_debq_22 , w1Bx_debq_23)",DEBQ: Emotional Eating,* assuming reverse scoring completed
29,DEBQ,w1debq_ext,"MEAN(w1Bx_debq_24 , w1Bx_debq_25 , w1Bx_debq_26 , w1Bx_debq_27 , w1Bx_debq_28 , w1Bx_debq_29 , w1Bx_debq_30 , w1Bx_debq_31 , w1Bx_debq_32 , w1Bx_debq_33)",DEBQ: External Eating,"* assuming reverse scoring completed, also beware these are out of order"


In [24]:
s1_behav.filter(like="debq", axis=1).head()

,w1Bx_debq_1,w1Bx_debq_2,w1Bx_debq_3,w1Bx_debq_4,w1Bx_debq_5,w1Bx_debq_6,w1Bx_debq_7,w1Bx_debq_8,w1Bx_debq_9,w1Bx_debq_10,w1Bx_debq_11,w1Bx_debq_12,w1Bx_debq_13,w1Bx_debq_14,w1Bx_debq_15,w1Bx_debq_16,w1Bx_debq_17,w1Bx_debq_18,w1Bx_debq_19,w1Bx_debq_20,w1Bx_debq_21,w1Bx_debq_22,w1Bx_debq_23,w1Bx_debq_24,w1Bx_debq_25,w1Bx_debq_26,w1Bx_debq_27,w1Bx_debq_28,w1Bx_debq_29,w1Bx_debq_30,w1Bx_debq_31,w1Bx_debq_32,w1Bx_debq_33
patID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
bbx_001,2,2,2,2,2,1,2,2,3,2,3,3,2,2,2,2,1,1,1,1,1,1,3,4,4,4,3,3,3,2,2,2,4
bbx_002,4,3,1,2,3,4,2,3,1,5,3,2,3,1,1,1,1,4,3,1,1,2,1,3,1,2,2,2,3,2,3,2,5
bbx_003,3,1,3,2,2,3,2,1,1,2,3,5,1,2,2,1,2,3,3,1,4,4,5,5,5,5,4,4,5,5,3,2,3
bbx_004,3,3,3,4,3,3,3,3,3,3,2,2,3,3,2,2,2,2,2,2,3,3,3,4,4,4,4,3,3,4,3,3,4
bbx_005,2,1,2,1,2,2,2,2,2,3,4,4,4,4,4,2,3,5,4,3,2,4,4,4,4,3,4,3,4,3,2,4,2


In [25]:
debq_df=debq_calculation()

,w1debq_tot,w1debq_r,w1debq_e,w1debq_ext
bbx_166,3.454545,3.3,3.230769,3.9
bbx_167,1.969697,2.0,1.153846,3.0
bbx_168,2.363636,1.8,2.000000,3.4
bbx_169,2.545455,2.9,2.307692,2.5
bbx_170,2.939394,2.7,2.615385,3.6



[INFO] completed DEBQ calculations.



## SPSRQ <a anchor="class" id="spsrq"></a>
___

In [26]:
# spsrq Calculations
scoring.loc[scoring['Set']=="SPSRQ"]

,Set,Output,Calculation,Description,Notes
30,SPSRQ,w1spsrq_tot,"SUM(w1Bx_spsrq1 , w1Bx_spsrq2 , w1Bx_spsrq3 , w1Bx_spsrq4 , w1Bx_spsrq5 , w1Bx_spsrq6 , w1Bx_spsrq7 , w1Bx_spsrq8 , w1Bx_spsrq9 , w1Bx_spsrq10 , w1Bx_spsrq11 , w1Bx_spsrq12 , w1Bx_spsrq13 , w1Bx_spsrq14 , w1Bx_spsrq15 , w1Bx_spsrq16 , w1Bx_spsrq17 , w1Bx_spsrq18 , w1Bx_spsrq19 , w1Bx_spsrq20)",SPSRQ: Total/Overall Score,NaN
31,SPSRQ,w1spsrq_sp,"SUM(w1Bx_spsrq1 , w1Bx_spsrq3 , w1Bx_spsrq5 , w1Bx_spsrq7 , w1Bx_spsrq9 , w1Bx_spsrq11 , w1Bx_spsrq13 , w1Bx_spsrq15 , w1Bx_spsrq17 , w1Bx_spsrq19)",SPSRQ: Sensitivity to Punishment,NaN
32,SPSRQ,w1spsrq_sr,"SUM(w1Bx_spsrq2 , w1Bx_spsrq4 , w1Bx_spsrq6 , w1Bx_spsrq8 , w1Bx_spsrq10 , w1Bx_spsrq12 , w1Bx_spsrq14 , w1Bx_spsrq16 , w1Bx_spsrq18 , w1Bx_spsrq20)",SPSRQ: Sensitivity to Reward,NaN


In [27]:
s1_behav.filter(like="spsrq", axis=1).head()

,w1Bx_spsrq1,w1Bx_spsrq2,w1Bx_spsrq3,w1Bx_spsrq4,w1Bx_spsrq5,w1Bx_spsrq6,w1Bx_spsrq7,w1Bx_spsrq8,w1Bx_spsrq9,w1Bx_spsrq10,w1Bx_spsrq11,w1Bx_spsrq12,w1Bx_spsrq13,w1Bx_spsrq14,w1Bx_spsrq15,w1Bx_spsrq16,w1Bx_spsrq17,w1Bx_spsrq18,w1Bx_spsrq19,w1Bx_spsrq20
patID,,,,,,,,,,,,,,,,,,,,
bbx_001,4.0,4.0,4.0,4.0,2.0,4.0,4.0,4.0,4.0,2.0,4.0,4.0,2.0,2.0,2.0,2.0,3.0,4.0,4.0,2.0
bbx_002,2.0,1.0,2.0,3.0,2.0,1.0,4.0,1.0,4.0,3.0,5.0,3.0,2.0,2.0,1.0,2.0,2.0,3.0,4.0,2.0
bbx_003,4.0,2.0,2.0,4.0,3.0,3.0,5.0,3.0,4.0,3.0,4.0,5.0,2.0,3.0,4.0,5.0,3.0,4.0,4.0,3.0
bbx_004,2.0,2.0,2.0,4.0,1.0,4.0,1.0,3.0,4.0,3.0,1.0,4.0,1.0,4.0,2.0,3.0,2.0,4.0,2.0,3.0
bbx_005,4.0,2.0,3.0,4.0,3.0,3.0,3.0,2.0,5.0,4.0,5.0,5.0,1.0,4.0,5.0,4.0,4.0,5.0,3.0,4.0


In [28]:
spsrq_df=spsrq_calculation()


[INFO] ERROR with SPSRQ calculation for ids:  {'bbx_057'}


,w1spsrq_tot,w1spsrq_sp,w1spsrq_sr
bbx_166,67.0,35.0,32.0
bbx_167,48.0,24.0,24.0
bbx_168,52.0,29.0,23.0
bbx_169,60.0,29.0,31.0
bbx_170,66.0,34.0,32.0



[INFO] completed SPSRQ calculations.



## BIS/BAS  <a anchor="class" id="bisbas"></a>
---

In [29]:
# debq Calculations
scoring.loc[scoring['Set']=="BIS/BAS"]

,Set,Output,Calculation,Description,Notes
33,BIS/BAS,w1bisbas_bastot,"SUM(w1Bx_bis_bas_3 , w1Bx_bis_bas_8 , w1Bx_bis_bas_11 , w1Bx_bis_bas_20, w1Bx_bis_bas_5 , w1Bx_bis_bas_9 , w1Bx_bis_bas_14 , w1Bx_bis_bas_19, w1Bx_bis_bas_4 , w1Bx_bis_bas_6 , w1Bx_bis_bas_13 , w1Bx_bis_bas_17 , w1Bx_bis_bas_22)",BIS/BAS: BAS Total Score,* assuming reverse scoring completed
34,BIS/BAS,w1bisbas_basd,"SUM(w1Bx_bis_bas_3 , w1Bx_bis_bas_8 , w1Bx_bis_bas_11 , w1Bx_bis_bas_20)",BIS/BAS: BAS Drive Score,* assuming reverse scoring completed
35,BIS/BAS,w1bisbas_basfs,"SUM(w1Bx_bis_bas_5 , w1Bx_bis_bas_9 , w1Bx_bis_bas_14 , w1Bx_bis_bas_19)",BIS/BAS: BAS Fun Seeking Score,* assuming reverse scoring completed
36,BIS/BAS,w1bisbas_basrr,"SUM(w1Bx_bis_bas_4 , w1Bx_bis_bas_6 , w1Bx_bis_bas_13 , w1Bx_bis_bas_17 , w1Bx_bis_bas_22)",BIS/BAS: BAS Reward Responsiveness Score,* assuming reverse scoring completed
37,BIS/BAS,w1bisbas_bis,"SUM(w1Bx_bis_bas_2 , w1Bx_bis_bas_7 , w1Bx_bis_bas_12 , w1Bx_bis_bas_15 , w1Bx_bis_bas_18 , w1Bx_bis_bas_21 , w1Bx_bis_bas_23)",BIS/BAS: BIS Score,* assuming reverse scoring completed


In [30]:
s1_behav.filter(like="bis_bas", axis=1).head()

,w1Bx_bis_bas_1,w1Bx_bis_bas_2,w1Bx_bis_bas_3,w1Bx_bis_bas_4,w1Bx_bis_bas_5,w1Bx_bis_bas_6,w1Bx_bis_bas_7,w1Bx_bis_bas_8,w1Bx_bis_bas_9,w1Bx_bis_bas_10,w1Bx_bis_bas_11,w1Bx_bis_bas_12,w1Bx_bis_bas_13,w1Bx_bis_bas_14,w1Bx_bis_bas_15,w1Bx_bis_bas_16,w1Bx_bis_bas_17,w1Bx_bis_bas_18,w1Bx_bis_bas_19,w1Bx_bis_bas_20,w1Bx_bis_bas_21,w1Bx_bis_bas_22,w1Bx_bis_bas_23
patID,,,,,,,,,,,,,,,,,,,,,,,
bbx_001,1,2,2,2,2,1,2,2,2,2,2,2,2,3,3,2,2,2,2,3,2,2,2
bbx_002,1,4,3,1,1,1,2,3,3,3,2,2,1,2,3,1,2,2,2,3,3,1,2
bbx_003,1,4,2,1,1,1,4,1,1,1,2,2,2,2,1,1,2,1,2,2,2,1,2
bbx_004,2,3,1,1,1,1,2,1,1,4,2,2,2,3,3,1,2,2,2,2,1,1,2
bbx_005,2,2,2,1,2,1,3,1,3,1,2,2,1,3,1,2,1,1,2,3,1,2,2


In [31]:
bis_bas_df=bis_bas_calculation()


[INFO] ERROR with BIS/BAS calculation for ids:  {'bbx_057'}


,w1bisbas_bastot,w1bisbas_basd,w1bisbas_basfs,w1bisbas_basrr,w1bisbas_bis
bbx_166,43.0,11.0,14.0,18.0,25.0
bbx_167,45.0,14.0,12.0,19.0,18.0
bbx_168,40.0,9.0,15.0,16.0,23.0
bbx_169,43.0,12.0,13.0,18.0,22.0
bbx_170,43.0,11.0,13.0,19.0,23.0



[INFO] completed BIS/BAS calculations.



## PFS  <a anchor="class" id="pfs"></a>
---

In [32]:
scoring.loc[scoring['Set']=="PFS"]

,Set,Output,Calculation,Description,Notes
38,PFS,w1pfs_tot,"MEAN(w1Bx_pfs_1 , w1Bx_pfs_3 , w1Bx_pfs_8 , w1Bx_pfs_16 , w1Bx_pfs_17 , w1Bx_pfs_19, w1Bx_pfs_5 , w1Bx_pfs_6 , w1Bx_pfs_10 , w1Bx_pfs_11,w1Bx_pfs_14 , w1Bx_pfs_15 , w1Bx_pfs_18 , w1Bx_pfs_20 , w1Bx_pfs_21)",PFS: Total Score,"*items 2, 4, 7, 9, 12, 13 omitted from scoring"
39,PFS,w1pfs_fa,"MEAN(w1Bx_pfs_1 , w1Bx_pfs_3 , w1Bx_pfs_8 , w1Bx_pfs_16 , w1Bx_pfs_17 , w1Bx_pfs_19)",PFS: Food Available,NaN
40,PFS,w1pfs_fp,"MEAN(w1Bx_pfs_5 , w1Bx_pfs_6 , w1Bx_pfs_10 , w1Bx_pfs_11)",PFS: Food Present,NaN
41,PFS,w1pfs_ft,"MEAN(w1Bx_pfs_14 , w1Bx_pfs_15 , w1Bx_pfs_18 , w1Bx_pfs_20 , w1Bx_pfs_21)",PFS: Food Tasted,NaN


In [33]:
s1_behav.filter(like="pfs", axis=1).head()

,w1Bx_pfs_1,w1Bx_pfs_2,w1Bx_pfs_3,w1Bx_pfs_4,w1Bx_pfs_5,w1Bx_pfs_6,w1Bx_pfs_7,w1Bx_pfs_8,w1Bx_pfs_9,w1Bx_pfs_10,w1Bx_pfs_11,w1Bx_pfs_12,w1Bx_pfs_13,w1Bx_pfs_14,w1Bx_pfs_15,w1Bx_pfs_16,w1Bx_pfs_17,w1Bx_pfs_18,w1Bx_pfs_19,w1Bx_pfs_20,w1Bx_pfs_21
patID,,,,,,,,,,,,,,,,,,,,,
bbx_001,3,2,2,2,1,1,3,1,3,2,2,2,1,2,2,2,1,2,2,2,2
bbx_002,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
bbx_003,4,4,4,4,4,5,5,3,5,5,5,4,4,4,5,2,2,3,4,2,4
bbx_004,3,2,2,1,2,1,3,2,2,2,2,2,1,2,2,1,1,2,2,1,2
bbx_005,4,2,2,2,1,3,5,5,4,3,5,2,3,2,2,4,5,3,5,3,2


In [34]:
 pfs_df=pfs_calculation()

,w1pfs_tot,w1pfs_fa,w1pfs_fp,w1pfs_ft
bbx_166,4.266667,4.333333,4.75,3.8
bbx_167,1.666667,1.000000,1.50,2.6
bbx_168,2.266667,1.166667,3.50,2.6
bbx_169,1.466667,1.500000,1.00,1.8
bbx_170,1.600000,1.500000,2.00,1.4



[INFO] completed PFS calculations.



## YFAS  <a anchor="class" id="yfas"></a>
---

In [35]:
scoring.loc[scoring['Set']=="YFAS"]

,Set,Output,Calculation,Description,Notes
42,YFAS,w1yfas_c1,"SUM(w1Bx_yfas_1 , w1Bx_yfas_2 , w1Bx_yfas_3)",YFAS: Criterion 1 Score - Substance taken in larger amount and for longer period than intended,*assuming weird scoring has been completed
43,YFAS,w1yfas_c2,"SUM(w1Bx_yfas_4 , w1Bx_yfas17_6 , w1Bx_yfas17_8 , w1Bx_yfas25)",YFAS: Criterion 2 Score - Persistent desire or repeated unsuccessful attempts to quit,*assuming weird scoring has been completed
44,YFAS,w1yfas_c3,"SUM(w1Bx_yfas_5 , w1Bx_yfas_6 , w1Bx_yfas_7)","YFAS: Criterion 3 Score - Much time/activity to obtain, use, recover",*assuming weird scoring has been completed
45,YFAS,w1yfas_c4,"SUM(w1Bx_yfas_8 , w1Bx_yfas_9 , w1Bx_yfas_10 , w1Bx_yfas_11)","YFAS: Criterion 4 Score - Important social, occupational, or recreational activities given up or reduced",*assuming weird scoring has been completed
46,YFAS,w1yfas_c5,w1Bx_yfas_17_3,"YFAS: Criterion 5 Score - Use continues despite knowledge of adverse consequences (e.g., failure to fulfill role obligation, use when physically hazardous)",*assuming weird scoring has been completed
47,YFAS,w1yfas_c6,"SUM(w1Bx_yfas17_4 , w1Bx_yfas17_5)",YFAS: Criterion 6 Score - Tolerance (marked increase in amount; marked decrease in effect),*assuming weird scoring has been completed
48,YFAS,w1yfas_c7,"SUM(w1Bx_yfas_12 , w1Bx_yfas_13 , w1Bx_yfas_14)",YFAS: Criterion 7 Score - Characteristic withdrawal symptoms; substance taken to relieve withdrawal,*assuming weird scoring has been completed
49,YFAS,w1yfas_c8,"SUM(w1Bx_yfas_15 , w1Bx_yfas_16)",YFAS: Criterion 8 Score - Use causes clinically significant impairment or distress,*assuming weird scoring has been completed
50,YFAS,w1yfas_c1_met,"if w1yfas_c1>=1, w1yfas_c1_met=1, else if w1yfas_c1=0, w1yfas_c1_met=0","YFAS: Criterion 1 Score Met: 1=Criterion Met, 0=Criterion Not Met",NaN
51,YFAS,w1yfas_c2_met,"if w1yfas_c2>=1, w1yfas_c2_met=1, else if w1yfas_c2=0, w1yfas_c2_met=0","YFAS: Criterion 2 Score Met: 1=Criterion Met, 0=Criterion Not Met",NaN


In [36]:
s1_behav.filter(like="yfas", axis=1).head()

,w1Bx_yfas_1,w1Bx_yfas_2,w1Bx_yfas_3,w1Bx_yfas_4,w1Bx_yfas_5,w1Bx_yfas_6,w1Bx_yfas_7,w1Bx_yfas_8,w1Bx_yfas_9,w1Bx_yfas_10,w1Bx_yfas_11,w1Bx_yfas_12,w1Bx_yfas_13,w1Bx_yfas_14,w1Bx_yfas_15,w1Bx_yfas_16,w1Bx_yfas_17_1,w1Bx_yfas_17_2,w1Bx_yfas_17_3,w1Bx_yfas_17_4,w1Bx_yfas_17_5,w1Bx_yfas_17_6,w1Bx_yfas_17_7,w1Bx_yfas_17_8,w1Bx_yfas_25,w1Bx_yfas_26,w1Bx_yfas_27
patID,,,,,,,,,,,,,,,,,,,,,,,,,,,
bbx_001,2,2,1,3,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,.,.
bbx_002,2,3,1,1,1,1,1,1,1,1,1,1,1,2,1,1,1,1,1,1,1,2,2,2,2,.,.
bbx_003,4,4,3,2,3,4,1,2,3,2,1,1,1,2,.,1,1,1,1,1,1,2,2,2,2,"1,2,4,6,8,15",.
bbx_004,2,3,2,3,1,1,3,1,2,1,1,2,2,3,2,1,1,1,1,1,1,2,2,2,4,"1,6",.
bbx_005,3,4,2,4,1,3,3,3,4,3,1,1,3,3,2,1,1,1,1,2,2,2,1,1,3,"2,4,6,8,25",.


In [37]:
yfas_calculation()


[INFO] ERROR with YFAS calculation for ids:  {'bbx_063', 'bbx_105', 'bbx_123', 'bbx_052', 'bbx_005', 'bbx_116', 'bbx_043', 'bbx_069', 'bbx_163', 'bbx_068', 'bbx_126', 'bbx_040', 'bbx_134', 'bbx_029', 'bbx_125', 'bbx_081', 'bbx_094', 'bbx_047', 'bbx_057', 'bbx_049', 'bbx_158', 'bbx_156', 'bbx_111', 'bbx_165', 'bbx_138'}


,w1yfas_c1,w1yfas_c2,w1yfas_c3,w1yfas_c4,w1yfas_c5,w1yfas_c6,w1yfas_c7,w1yfas_c8,w1yfas_c1_met,w1yfas_c2_met,w1yfas_c3_met,w1yfas_c4_met,w1yfas_c5_met,w1yfas_c6_met,w1yfas_c7_met,w1yfas_c8_met,w1yfas_cont_score
bbx_151,1.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,3.0
bbx_152,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
bbx_153,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
bbx_154,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
bbx_155,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
bbx_156,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
bbx_157,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
bbx_158,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
bbx_159,2.0,3.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,4.0
bbx_160,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0



[INFO] completed YFAS calculations.



## IPAQ  <a anchor="class" id="ipaq"></a>

--- 


In [38]:
""" 
# IN PROGRESS
"""

' \n# IN PROGRESS\n'

## Questions 

- What to do with nan values?


---